🌍 ───────────────────────────────────────────────────────── 🌍

*                       THERESA ROCCO PEREIRA BARBOSA
*                 Brazilian Ambientalist | Geologist and Soil Scientist                  
*               imakemapas@outlook.com.br | theresa.rocco@ufrrj.br | +55 24 998417085

🌍 ───────────────────────────────────────────────────────── 🌍

In [ ]:
%reset

# Inteligência Territorial - Cadastro Ambiental Rural

**Este código representa o resultado de um estudo em constante evolução e aprimoramento. Seu objetivo não é ser definitivo, mas sim um ponto de partida. Sugestões e críticas construtivas são não apenas bem-vindas, mas essenciais para o seu aperfeiçoamento.**

**O propósito deste trabalho é realizar o processamento, a integração e a análise ambiental de dados do Cadastro Ambiental Rural (CAR), além de outras bases geoespaciais disponíveis para o território brasileiro. O foco principal é apoiar análises e auditorias ambientais com base nos critérios estabelecidos pelo Código Florestal Brasileiro.**

Este tutorial utiliza prioritariamente o módulo **arcpy*** e adota o formato **File Geodatabase (GDB)** para armazenamento e manipulação dos dados espaciais. 

O GDB foi escolhido devido às suas características, como: compactação eficiente que reduz o uso de memória e espaço em disco, suporte a grandes volumes de dados, maior velocidade nas operações, e a capacidade de armazenar estruturas complexas, como topologias e relações entre tabelas.

No futuro pretendo disponibilizar versão utilizando bibliotecas gratuitas como GeoPandas, Pandas e Numpy em Python e/ou terra, sp, dplyr em R.

**Licença ArcGIS: Laboratório de Modelagem e Evolução Geológica da Universidade Federal Rural do Rio de Janeiro*

# Iniciando Ambiente de Trabalho

#### Importar Bibliotecas

In [ ]:
import arcpy
import os
import zipfile

from arcpy import env
from arcpy.sa import *

arcpy.env.overwriteOutput = True

#### Definir o Caminho do Geodatabase & Definir o Ambiente de Trabalho

In [ ]:
GDB = r"C:\CAR_2025\CAR_PY\CAR_2025.gdb"
arcpy.env.workspace = GDB

#### Verificar a Existência do Geodatabase & Criar o Geodatabase Caso Não Exista

In [ ]:
if not arcpy.Exists(GDB):
    arcpy.CreateFileGDB_management(r"D:\CAR_2025\CAR_PY", 
                                   "CAR_2025f.gdb")
    print("Geodatabase criado com sucesso!")
else:
    print("Geodatabase já existe!")

In [ ]:
# Verifique as feature classes no workspace
feature_classes = arcpy.ListFeatureClasses()

# Exiba todas as feature classes no workspace
print(feature_classes)

# Data Wrangling

## Dados Uso e Cobertura do Solo - MaBiomas, ano 2023

(Link Download Direto - Coleção 9) https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_9/lclu/coverage/brasil_coverage_2023.tif

In [ ]:
MB = "brasil_coverage_23"

In [ ]:
crsMB  = arcpy.Describe(MB).spatialReference
print("CRS MB:", crsMB.name)

cell_size_x = arcpy.GetRasterProperties_management(MB, "CELLSIZEX").getOutput(0)
cell_size_y = arcpy.GetRasterProperties_management(MB, "CELLSIZEY").getOutput(0)
print(f"Resolução (X): {cell_size_x}")
print(f"Resolução (Y): {cell_size_y}")

#### Reclassificar

| **Código** | **Categoria**              | **Descrição**                          |
|------------|----------------------------|----------------------------------------|
| **1**      | Vegetação Nativa           | Forest Formation (3)                  |
|            |                            | Savanna Formation (4)                 |
|            |                            | Mangrove (5)                          |
|            |                            | Floodable Forest (6)                  |
|            |                            | Wooded Sandbank Vegetation (49)       |
|            |                            | Wetland (11)                          |
|            |                            | Grassland (12)                        |
|            |                            | Hypersaline Tidal Flat (32)           |
|            |                            | Herbaceous Sandbank Vegetation (50)   |
| **2**      | Corpo d'Água               | Water (26)                            |
|            |                            | River, Lake and Ocean (33)            |
| **3**      | Soja                       | Soybean (39)                          |
| **4**      | Pastagem                   | Pasture (15)                          |
| **5**      | Produtivo Agro             | Mosaic of Uses (21)                   |
|            |                            | Sugar Cane (20)                       |
|            |                            | Rice (40)                             |
|            |                            | Cotton (Beta) (62)                    |
|            |                            | Other Temporary Crops (41)            |
|            |                            | Coffee (46)                           |
|            |                            | Citrus (47)                           |
|            |                            | Palm Oil (35)                         |
|            |                            | Forest Plantation (9)                 |
|            |                            | Other Perennial Crops (48)            |
| **6**      | Outros                     | Rocky Outcrop (29)                    |
|            |                            | Beach, Dune, and Sand Spot (23)       |
|            |                            | Urban Area (24)                       |
|            |                            | Other non Vegetated Areas (25)        |
|            |                            | Mining (30)                           |
|            |                            | Aquaculture (31)                      |

In [ ]:
# Tabela de reclassificação
reclassification = RemapValue([
    [0, "NODATA"], 
    [3, 1], [4, 1], [5, 1], [6, 1], [11, 1], [12, 1], [32, 1], [49, 1], [50, 1],
    [33, 2],
    [39, 3],
    [15, 4], 
    [9, 5],  [20, 5], [21, 5], [35, 5], [62, 5],  
    [40, 5], [41, 5], [46, 5], [47, 5], [48, 5],
    [23, 6], [24, 6], [25, 6], [29, 5], [30, 6], [31, 6]
])

# 0 será convertido para NODATA.
# 3, 4, 5, 6, 11, 12, 32, 49, 50 serão convertidos para 1.
# 33 será convertido para 2.
# 39 será convertido para 3.
# 15 será convertido para 4.
# 9, 20, 21, 35, 40, 41, 46, 47, 48, 62 serão convertidos para 5.
# 23, 24, 25, 29, 30, 31 serão convertidos para 6.

In [ ]:
MBreclass = Reclassify(MB, "Value", reclassification, "NODATA")

In [ ]:
# Salve raster
MBreclass.save(f"{GDB}\\brasil_coverage_23_reclass")

#### Reprojetar

In [ ]:
MB = "brasil_coverage_23_reclass"

In [ ]:
albers_brasil = arcpy.SpatialReference()
albers_brasil.loadFromString("""
PROJCS["Conica_Equivalente_de_Albers_Brasil",
    GEOGCS["GCS_SIRGAS2000",
        DATUM["D_SIRGAS2000",
            SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295]],
    PROJECTION["Albers"],
    PARAMETER["standard_parallel_1",-2],
    PARAMETER["standard_parallel_2",-22],
    PARAMETER["latitude_of_origin",-12],
    PARAMETER["central_meridian",-54],
    PARAMETER["false_easting",5000000],
    PARAMETER["false_northing",10000000],
    UNIT["Meter",1]]
""")

In [ ]:
arcpy.management.ProjectRaster(
    in_raster       =  MB,
    out_raster      = "brasil_coverage_23_reclass_prj",
    out_coor_system =  albers_brasil,
    resampling_type =  "NEAREST",
    cell_size       = "30"
)

#### Raster para Poligono

In [ ]:
MB = "brasil_coverage_23_reclass_prj"

In [ ]:
arcpy.conversion.RasterToPolygon(MB, 
                                 "brasil_coverage_23_reclass_prj_POL", 
                                 simplify                  = "NO_SIMPLIFY", 
                                 raster_field              = "Value", 
                                 create_multipart_features = "SINGLE_OUTER_PART" #"MULTIPLE_OUTER_PART"
                                )

In [ ]:
MBp = "brasil_coverage_23_reclass_prj_POL"

## Dados PRODES: Desmatamento

https://terrabrasilis.dpi.inpe.br/downloads/ (acesso: 24/01/2025)

### [Prodes Brasil - até 2023](https://terrabrasilis.dpi.inpe.br/geonetwork/srv/eng/catalog.search#/metadata/fe02f2bf-2cc0-49d5-ab72-a3954f997408)

#### Estrutura

In [ ]:
PRODES_BR = "prodes_brasil_23"

In [ ]:
crsPRODES_BR  = arcpy.Describe(PRODES_BR).spatialReference
print("CRS PRODES :", crsPRODES_BR.name)

In [ ]:
cell_size_x = arcpy.GetRasterProperties_management(PRODES_BR, "CELLSIZEX").getOutput(0)
cell_size_y = arcpy.GetRasterProperties_management(PRODES_BR, "CELLSIZEY").getOutput(0)
print(f"Resolução (X): {cell_size_x}")
print(f"Resolução (Y): {cell_size_y}")

#### Reclassify

In [ ]:
# Tabela de reclassificação
reclassification = RemapValue([
[0,  2000],
[2,  2002],
[4,  2004],
[6,  2006],
[7,  2007],
[8,  2008],
[9,  2009],
[10, 2010],
[11, 2011],
[12, 2012],
[13, 2013],
[14, 2014],
[15, 2015],
[16, 2016],
[17, 2017],
[18, 2018],
[19, 2019],
[20, 2020],
[21, 2021],
[22, 2022],
[23, 2023],
[50, 2010],
[51, 2011],
[52, 2012],
[53, 2013],
[54, 2014],
[55, 2015],
[56, 2016],
[57, 2017],
[58, 2018],
[59, 2019],
[60, 2020],
[61, 2021],
[62, 2022],
[63, 2023],
[91,  "NODATA"],
[99,  "NODATA"],
[100, "NODATA"],
[101, "NODATA"]
])

In [ ]:
PRODESreclass = Reclassify(PRODES_BR, "Value", reclassification, "NODATA")

In [ ]:
# Salve o raster reclassificado
PRODESreclass.save(f"{GDB}\\prodes_brasil_23_reclass")

#### Reprojetar

In [ ]:
PRODES_BR = "prodes_brasil_23_reclass"

In [ ]:
#albers_brasil = arcpy.Describe(MB).spatialReference

albers_brasil = arcpy.SpatialReference()
albers_brasil.loadFromString("""
PROJCS["Conica_Equivalente_de_Albers_Brasil",
    GEOGCS["GCS_SIRGAS2000",
        DATUM["D_SIRGAS2000",
            SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295]],
    PROJECTION["Albers"],
    PARAMETER["standard_parallel_1",-2],
    PARAMETER["standard_parallel_2",-22],
    PARAMETER["latitude_of_origin",-12],
    PARAMETER["central_meridian",-54],
    PARAMETER["false_easting",5000000],
    PARAMETER["false_northing",10000000],
    UNIT["Meter",1]]
""")

In [ ]:
arcpy.management.ProjectRaster(
    in_raster       =  PRODES_BR,
    out_raster      = "prodes_brasil_23_reclass_prj",
    out_coor_system =  albers_brasil,
    resampling_type =  "NEAREST",
    cell_size       = "30"
)

#### Raster para Poligono

In [ ]:
PRODES_BR = "prodes_brasil_23_reclass_prj"

In [ ]:
arcpy.conversion.RasterToPolygon(PRODES_BR, 
                                 "prodes_brasil_23_reclass_prj_POL", 
                                 simplify                  = "NO_SIMPLIFY", 
                                 raster_field              = "Value", 
                                 create_multipart_features = "SINGLE_OUTER_PART" #"MULTIPLE_OUTER_PART"
                                )

In [ ]:
PRODES_BRp = "prodes_brasil_23_reclass_prj_POL"

### [Prodes Cerrado - 2024](https://terrabrasilis.dpi.inpe.br/geonetwork/srv/eng/catalog.search#/metadata/fe02f2bf-2cc0-49d5-ab72-a3954f997408)

#### Estrutura

In [ ]:
PRODES_CER = "prodes_cerrado_24"

In [ ]:
crsPRODES_CER = arcpy.Describe(PRODES_CER).spatialReference
print("CRS MB :", crsPRODES_CER.name)

In [ ]:
cell_size_x = arcpy.GetRasterProperties_management(PRODES_CER, "CELLSIZEX").getOutput(0)
cell_size_y = arcpy.GetRasterProperties_management(PRODES_CER, "CELLSIZEY").getOutput(0)
print(f"Resolução (X): {cell_size_x}")
print(f"Resolução (Y): {cell_size_y}")

#### Extract by Attributes

In [ ]:
where_clause = "VALUE = 24 Or VALUE = 64"

In [ ]:
extracted_raster = arcpy.sa.ExtractByAttributes(
    in_raster    = PRODES_CER, 
    where_clause = where_clause)

In [ ]:
extracted_raster.save("prodes_cerrado_24_filter")

#### Reclassify

In [ ]:
PRODES_CER = "prodes_cerrado_24_filter"

In [ ]:
reclassification = RemapValue([
[24,  2024],
[64,  2024]
])

In [ ]:
PRODES_CER_reclass = Reclassify(PRODES_CER, "Value", reclassification, "NODATA")

In [ ]:
PRODES_CER_reclass.save(f"{GDB}\\prodes_cerrado_24_filter_reclass")

#### Reporjetar

In [ ]:
PRODES_CER = "prodes_brasil_23_filter_reclass"

In [ ]:
arcpy.management.ProjectRaster(
    in_raster       =  PRODES_CER,
    out_raster      = "prodes_cerrado_24_filter_reclass_prj",
    out_coor_system =  albers_brasil,
    resampling_type =  "NEAREST",
    cell_size       = "30")

#### Raster para Polígono

In [ ]:
PRODES_CER = "prodes_cerrado_24_filter_reclass_prj"

In [ ]:
arcpy.conversion.RasterToPolygon(PRODES_CER, 
                                 "prodes_cerrado_24_filter_reclass_prj_POL", 
                                 simplify                  = "NO_SIMPLIFY", 
                                 raster_field              = "Value", 
                                 create_multipart_features = "SINGLE_OUTER_PART" #"MULTIPLE_OUTER_PART"
                                )

In [ ]:
PRODES_CERp = "prodes_cerrado_24_filter_reclass_prj_POL"

### Prodes Amazônia Legal - [cenas prioritárias 2024](https://terrabrasilis.dpi.inpe.br/geonetwork/srv/eng/catalog.search#/metadata/54c9ad38-cd5f-4629-bb42-aa75c205c010)

#### Prodes AL cenas prioritárias 2024

In [ ]:
PRODES_AL =  "prodes_amzlegal_prior24_prj"

### Prodes Único - Final

#### <mark>Onde Paramos?</mark>

In [ ]:
PRODES_ALp  =  "prodes_amzlegal_prior24_prj"
PRODES_BRp  = "prodes_brasil_23_reclass_prj_POL"
PRODES_CERp = "prodes_cerrado_24_filter_reclass_prj_POL"

#### Merge

In [ ]:
arcpy.management.Merge(inputs = [PRODES_BRp, PRODES_CERp, PRODES_ALp], 
                       output = "prodes_final")

In [ ]:
PRODES = "prodes_final"

#### Delete

In [ ]:
arcpy.DeleteField_management(PRODES, ["ID"])

## Dados Alerta MapBiomas

(Acesso 26/01/2024) https://plataforma.alerta.mapbiomas.org/downloads 

### Descompactar Alertas

In [ ]:
# Diretórios de entrada e saída
diretorio_entrada = r"D:\CAR_2025\CAR_R\dados_brutos\alerta_embargos_26_01_2025"
diretorio_saida   = r"D:\CAR_2025\CAR_R\dados_limpos\alerta_embargos_26_01_2025"

# Criar o diretório de saída caso não exista
if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)

In [ ]:
# Lista todos os arquivos .zip no diretório de entrada
arquivos_zip = [f for f in os.listdir(diretorio_entrada) if f.endswith(".zip")]
print(arquivos_zip)

In [ ]:
arquivo_zip = "dashboard_alerts-shapefile.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "dashboard_alerts-shapefile.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

### Reprojetar e Importar no GDB

In [ ]:
CAR = "CAR_prj"
albers_brasil = arcpy.Describe(CAR).spatialReference

arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "dashboard_alerts-shapefile.shp"),
    out_dataset     = os.path.join(GDB, "ALERTAprj"),
    out_coor_system = albers_brasil
)

### Corrigir Geometrias

In [ ]:
ALERTA = "ALERTAprj"

In [ ]:
arcpy.management.RepairGeometry(ALERTA,
                                delete_null       = "DELETE_NULL",
                                validation_method = "ESRI")

### Alertas 2024

In [ ]:
sql_ALERTA = "ANODETEC = 2024"

arcpy.conversion.ExportFeatures(ALERTA, 
                                f"{GDB}/ALERTAprj_2024", 
                                sql_ALERTA, 
                                "NOT_USE_ALIAS")

## Dados Embargos

In [ ]:
# Diretórios de entrada e saída
diretorio_entrada = r"D:\CAR_2025\CAR_R\dados_brutos\alerta_embargos_26_01_2025"
diretorio_saida   = r"D:\CAR_2025\CAR_R\dados_limpos\alerta_embargos_26_01_2025"

# Criar o diretório de saída caso não exista
if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)

In [ ]:
# Lista todos os arquivos .zip no diretório de entrada
arquivos_zip = [f for f in os.listdir(diretorio_entrada) if f.endswith(".zip")]
print(arquivos_zip)

In [ ]:
CAR = "CAR_prj"
albers_brasil = arcpy.Describe(CAR).spatialReference

### IBAMA

(Acesso 26/01/2025) https://dados.gov.br/dados/conjuntos-dados/termos-de-embargo

#### Descompactar

In [ ]:
arquivo_zip = "adm_embargo_ibama_a.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
CAR = "CAR_prj"
albers_brasil = arcpy.Describe(CAR).spatialReference

arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "adm_embargo_ibama_a.shp"),
    out_dataset     = os.path.join(GDB, "EMBIBprj"),
    out_coor_system = albers_brasil
)

#### Ajuste Colunas

In [ ]:
EMBIBprj = "EMBIBprj"

#---------------------------------
arcpy.management.AddField(EMBIBprj,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBIBprj,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBIBprj,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBIBprj,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBIBprj,
                                "EMB", 
                                "'EMB'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBIBprj,
                                "EMBfonte", 
                                f"'IBAMA - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
arcpy.management.AlterField(EMBIBprj, "EMBinfo", field_length=1000)

In [ ]:
arcpy.management.CalculateField(
    EMBIBprj,
    "EMBinfo",
    "'PROC: ' + !num_proces! + '; AUTUADO: ' + !nome_embar!.upper() + '; CPF_CNPJ: ' + !cpf_cnpj_e! + '; DESC: ' + !des_tad!.upper() + '; DATA: ' + !dat_embarg! + ' - ' + !dat_ult_al!",
    "PYTHON3"
)

In [ ]:
with arcpy.da.UpdateCursor(EMBIBprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"IBAMA_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
campos_a_manter  = ["OBJECTID_1", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBIBprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBIBprj, campos_a_remover)

### ICMBIO

(Acesso 26/01/2025) https://www.gov.br/icmbio/pt-br/assuntos/dados_geoespaciais/mapa-tematico-e-dados-geoestatisticos-das-unidades-de-conservacao-federais

#### Descompactar

In [ ]:
arquivo_zip = "embargos_icmbio_shp.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "embargos_icmbio.shp"),
    out_dataset     = os.path.join(GDB, "EMBICprj"),
    out_coor_system = albers_brasil
)

#### Ajuste Colunas

In [ ]:
EMBICprj = "EMBICprj"

#---------------------------------
arcpy.management.AddField(EMBICprj,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBICprj,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBICprj,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBICprj,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBICprj,
                                "EMB", 
                                "'EMBIC'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBICprj,
                                "EMBfonte", 
                                f"'ICMBIO - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
arcpy.management.AlterField(EMBICprj, "EMBinfo", field_length=1000)

In [ ]:
arcpy.management.CalculateField(
    EMBICprj,
    "EMBinfo",
    "'PROC: ' + !processo! + '; AUTUADO: ' + !autuado!.upper() + '; CPF_CNPJ: ' + !cpf_cnpj! + '; DESC: ' + !desc_sanc!.upper() + '; DATA: ' + (!data!.strftime('%d/%m/%Y') if !data! else 'N/D')",
    "PYTHON3"
)

In [ ]:
with arcpy.da.UpdateCursor(EMBICprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"ICMBIO_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
campos_a_manter  = ["OBJECTID", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBICprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBICprj, campos_a_remover)

### SEMA-MT

(Acesso 26/01/2025) https://geoportal.sema.mt.gov.br

#### Descompactar

In [ ]:
arquivo_zip = "AREAS_EMBARGADAS_SEMA.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "AREA_EMBARGADA_SIGA_POLIGONO.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "AREAS_EMBARGADAS_SEMA.shp"),
    out_dataset     = os.path.join(GDB, "EMBMTprj1"),
    out_coor_system = albers_brasil
)

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "AREA_EMBARGADA_SIGA_POLIGONO.shp"),
    out_dataset     = os.path.join(GDB, "EMBMTprj2"),
    out_coor_system = albers_brasil
)

#### Ajuste Colunas

In [ ]:
EMBMTprj1 = "EMBMTprj1"

#---------------------------------
arcpy.management.AddField(EMBMTprj1,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBMTprj1,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBMTprj1,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBMTprj1,
                          "EMBfonte",
                          "TEXT")

In [ ]:
EMBMTprj2 = "EMBMTprj2"

#---------------------------------
arcpy.management.AddField(EMBMTprj2,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBMTprj2,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBMTprj2,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBMTprj2,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBMTprj1,
                                "EMB", 
                                "'EMBMT'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBMTprj1,
                                "EMBfonte", 
                                f"'SEMA-MT - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBMTprj2,
                                "EMB", 
                                "'EMBMT'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBMTprj2,
                                "EMBfonte", 
                                f"'SEMA-MT - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
arcpy.management.AlterField(EMBMTprj1, "EMBinfo", field_length=1000)

In [ ]:
#---------------------------------
arcpy.management.CalculateField(
    EMBMTprj1,
    "EMBinfo",
    "'PROC: '+!N_PROCESSO!+'; AUTUADO: '+!NOME!+' - '+!PROPRIEDAD!+'; CPF_CNPJ:'+!CPF_CNPJ!+'; DESC: '+ !DANO!'; DATA: ' + !DAT_LAVRAT!",
    "PYTHON3"
)

In [ ]:
arcpy.management.AlterField(EMBMTprj2, "EMBinfo", field_length=1000)

In [ ]:
arcpy.management.CalculateField(
    EMBMTprj2,
    "EMBinfo",
    "'PROC: ' + !NUMERO_PRO! + '; AUTUADO: ' + !NOME_RAZAO! + ' - ' + !NOME_FANTA! + '; CPF_CNPJ: ' + !CPFCNPJ! + '; DESC: ' + !DESCRICAO_! + '; DATA: ' + !DATA_DO_AU!.strftime('%d/%m/%Y')",
    "PYTHON3"
)

#### Merge Final

In [ ]:
arcpy.management.Merge(inputs = [EMBMTprj1, EMBMTprj2], 
                       output = "EMBMTprj")

In [ ]:
EMBMTprj = "EMBMTprj"

In [ ]:
campos_a_manter  = ["OBJECTID_1", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBMTprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBMTprj, campos_a_remover)

In [ ]:
with arcpy.da.UpdateCursor(EMBMTprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"SEMA_MT_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

### SEMA-GO

(Acesso 26/01/2025) https://siga.meioambiente.go.gov.br/#/

#### Descompactar

In [ ]:
arquivo_zip = "Embargos Estaduais polígono - INÃ_SGA - público.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "13007192-6500-4935-bc26-78edfcc8dbe0.zip"
caminho_zip = os.path.join(diretorio_saida, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "vw_embargo_ina_sga_poligono_publico.shp"),
    out_dataset     = os.path.join(GDB, "EMBGOprj"),
    out_coor_system = albers_brasil
)

#### Ajuste Colunas

In [ ]:
EMBGOprj = "EMBGOprj"

#---------------------------------
arcpy.management.AddField(EMBGOprj,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBGOprj,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBGOprj,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBGOprj,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBGOprj,
                                "EMB", 
                                "'EMBGO'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBGOprj,
                                "EMBfonte", 
                                f"'SEMA-GO - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
with arcpy.da.UpdateCursor(EMBGOprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"SEMA_GO_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
arcpy.management.AlterField(EMBGOprj, "EMBinfo", field_length=1000)

In [ ]:
arcpy.management.CalculateField(
    EMBGOprj,
    "EMBinfo",
    "'PROC: ' + !codigo_pro! + '; AUTUADO: ' + !nome_razao! + '; DESC: ' + !categoria!.upper() + ' DATA: ' + !data_criac!.strftime('%d/%m/%Y')",
    "PYTHON3"
)

In [ ]:
campos_a_manter  = ["OBJECTID", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBGOprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBGOprj, campos_a_remover)

### SEMA-PA

(Acesso 26/01/2025) https://monitoramento.semas.pa.gov.br/ldi/

#### Descompactar

In [ ]:
arquivo_zip = "LDI_publicacao_automatizado.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "LDI_publicacao_manual.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "LDI_publicacao_sem_sobresicao.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "LDI_publicacao_automatizado.shp"),
    out_dataset     = os.path.join(GDB, "EMBPAprj1"),
    out_coor_system = albers_brasil
)

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "LDI_publicacao_manual.shp"),
    out_dataset     = os.path.join(GDB, "EMBPAprj2"),
    out_coor_system = albers_brasil
)

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "LDI_publicacao_sem_sobresicao.shp"),
    out_dataset     = os.path.join(GDB, "EMBPAprj3"),
    out_coor_system = albers_brasil
)

#### Ajustes Colunas

In [ ]:
EMBPAprj1 = "EMBPAprj1"

#---------------------------------
arcpy.management.AddField(EMBPAprj1,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBPAprj1,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBPAprj1,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBPAprj1,
                          "EMBfonte",
                          "TEXT")

In [ ]:
EMBPAprj2 = "EMBPAprj2"

#---------------------------------
arcpy.management.AddField(EMBPAprj2,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBPAprj2,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBPAprj2,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBPAprj2,
                          "EMBfonte",
                          "TEXT")

In [ ]:
EMBPAprj3 = "EMBPAprj3"

#---------------------------------
arcpy.management.AddField(EMBPAprj3,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBPAprj3,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBPAprj3,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBPAprj3,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBPAprj1,
                                "EMB", 
                                "'EMBPA'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBPAprj1,
                                "EMBfonte", 
                                f"'SEMA-PA - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBPAprj2,
                                "EMB", 
                                "'EMBPA'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBPAprj2,
                                "EMBfonte", 
                                f"'SEMA-PA - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBPAprj3,
                                "EMB", 
                                "'EMBPA'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBPAprj3,
                                "EMBfonte", 
                                f"'SEMA-PA - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
arcpy.management.AlterField(EMBPAprj1, "EMBinfo", field_length=1000)
arcpy.management.AlterField(EMBPAprj2, "EMBinfo", field_length=1000)
arcpy.management.AlterField(EMBPAprj3, "EMBinfo", field_length=1000)

In [ ]:
#---------------------------------
arcpy.management.CalculateField(
    EMBPAprj1,
    "EMBinfo",
    "'PROC: '+!P_SemCAR!+!P_ComCAR!+'; COD: '+!codigo!+'; COD_LDI: '+!codLdi!+'; DESC: '+!Status!+' '+!NUM_CAR! +'; DATA: '+!Dat_Depois!.strftime('%d/%m/%Y')",
    "PYTHON3"
)

In [ ]:
#---------------------------------
arcpy.management.CalculateField(
    EMBPAprj2,
    "EMBinfo",
    "'PROC: '+!P_SemCAR_1!+!P_ComCAR_1!+'; DESC: '+!Status! +'; DATA: ' + (!Dat_Depois!.strftime('%d/%m/%Y') if !Dat_Depois! else 'N/D')",
    "PYTHON3"
)

In [ ]:
#---------------------------------
arcpy.management.CalculateField(
    EMBPAprj3,
    "EMBinfo",
    "'PROC: '+!P_SemCAR!+!P_ComCAR!+'; COD: '+!codigo!+'; COD_LDI: '+!codLdi!+'; DESC: '+!Status!+' '+!NUM_CAR! +'; DATA: '+ (!Dat_Depois!.strftime('%d/%m/%Y') if !Dat_Depois! else 'N/D')",
    "PYTHON3"
) 

#### Merge Final

In [ ]:
arcpy.management.Merge(inputs = [EMBPAprj1, EMBPAprj2, EMBPAprj3], 
                       output = "EMBPAprj")

In [ ]:
EMBPAprj = "EMBPAprj"

In [ ]:
campos_a_manter  = ["OBJECTID", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBPAprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBPAprj, campos_a_remover)

In [ ]:
with arcpy.da.UpdateCursor(EMBPAprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"SEMA_PA_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

### IPAAM-AM

(Acesso 26/01/2025) https://gis.ipaam.am.gov.br/geoportal/

#### Descompactar

In [ ]:
arquivo_zip = "Areas_Embargadas__IPAAM__1738076417598.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

In [ ]:
arquivo_zip = "Areas_Embargadas__IPAAM___IPAAM__1738076428658.zip"
caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)

# Verifica se o arquivo existe antes de tentar descompactar
if os.path.exists(caminho_zip):
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_saida)
        print(f"Arquivo '{arquivo_zip}' descompactado com sucesso em: {diretorio_saida}")
else:
    print(f"O arquivo '{arquivo_zip}' não foi encontrado em: {diretorio_entrada}")

#### Reprojetar e Importar no GDB

In [ ]:
arcpy.Project_management(
    in_dataset      = os.path.join(diretorio_saida, "_reas_Embargadas__IPAAM___IPAAM_.shp"),
    out_dataset     = os.path.join(GDB, "EMBAMprj"),
    out_coor_system = albers_brasil
)

#### Ajuste Colunas

In [ ]:
EMBAMprj = "EMBAMprj"

#---------------------------------
arcpy.management.AddField(EMBAMprj,
                          "EMB",
                          "TEXT")

arcpy.management.AddField(EMBAMprj,
                          "EMBid",
                          "TEXT")

arcpy.management.AddField(EMBAMprj,
                          "EMBinfo",
                          "TEXT")

arcpy.management.AddField(EMBAMprj,
                          "EMBfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(EMBAMprj,
                                "EMB", 
                                "'EMBAM'", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(EMBAMprj,
                                "EMBfonte", 
                                f"'IPAAM-AM - {datetime.datetime.today().strftime('%d/%m/%Y')}'", 
                                "PYTHON3")

In [ ]:
arcpy.management.AlterField(EMBAMprj, "EMBinfo", field_length=1000)

In [ ]:
arcpy.management.CalculateField(
    EMBAMprj,
    "EMBinfo",
    "'PROC: ' + (!cod_origem! if !cod_origem! else 'N/D') + ' - ' + (!n_termo! if !n_termo! else 'N/D') + '; AUTUADO: ' + (!nom_pessoa! if !nom_pessoa! else 'ND') + '; DESC: ' +!des_termo! + ' - ' + !status_ter! + ' DATA: ' + (!data_cadas! if !data_cadas! else !data_termo!)",
    "PYTHON3"
)

In [ ]:
with arcpy.da.UpdateCursor(EMBAMprj, ["EMBid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"IPAAM-AM_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
campos_a_manter  = ["OBJECTID", "Shape", "EMBid", "EMBinfo", "EMBfonte", "EMB", "Shape_Length", "Shape_Area"]
todos_os_campos  = [field.name for field in arcpy.ListFields(EMBAMprj)]
campos_a_remover = [campo for campo in todos_os_campos if campo not in campos_a_manter]

arcpy.management.DeleteField(EMBAMprj, campos_a_remover)

### EMBARGOS FINAL

In [ ]:
EMBIBprj = "EMBIBprj"
EMBICprj = "EMBICprj"
EMBMTprj = "EMBMTprj"
EMBPAprj = "EMBPAprj"
EMBGOprj = "EMBGOprj"
EMBAMprj = "EMBAMprj"

In [ ]:
arcpy.management.Merge(inputs = [EMBIBprj,EMBICprj,EMBMTprj,EMBPAprj,EMBGOprj,EMBAMprj], 
                       output = "EMBfinalprj")

In [ ]:
EMBfinalprj = "EMBfinalprj"

In [ ]:
arcpy.management.RepairGeometry(EMBfinalprj,
                                delete_null       = "DELETE_NULL",
                                validation_method = "ESRI")

## Dados CAR

A construção de uma base consolidada do CAR para todo o território brasileiro começa com o acesso aos dados do perímetro dos imóveis rurais cadastrados no sistema. 

Há dois caminhos principais para obter esses dados: 

* [serviço geoespacial WFS do CAR](https://geoserver.car.gov.br/geoserver/wfs) ou
* [site oficial do SICAR](https://consultapublica.car.gov.br/publico/estados/downloads), onde os arquivos estão organizados por unidade federativa.

Neste tutorial, optaremos pela segunda alternativa, que resulta no download de 27 pastas compactadas, cada uma contendo os limites do CAR para uma unidade federativa.

Acesso: 07 de janeiro, 2025

#### Descompactar CAR

Após baixar os arquivos compactados dos 27 Estados, o próximo passo é descompactá-los e organizá-los com nomes padronizados, preparando-os para a integração posterior. Os dados baixados estão armazenados na pasta indicada por `diretorio_entrada`, enquanto os arquivos descompactados e renomeados serão salvos na pasta especificada por `diretorio_saida`.

In [ ]:
# Diretórios de entrada e saída
diretorio_entrada = r"D:\CAR_2025\CAR_R\dados_brutos\CAR_zipped"
diretorio_saida   = r"D:\CAR_2025\CAR_R\dados_limpos\CAR_unzipped2"

# Criar o diretório de saída caso não exista
if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)

In [ ]:
# Lista todos os arquivos .zip no diretório de entrada
arquivos_zip = [f for f in os.listdir(diretorio_entrada) if f.endswith(".zip")]
print(arquivos_zip)

In [ ]:
# Itera sobre cada arquivo .zip encontrado
for arquivo_zip in arquivos_zip:
    caminho_zip = os.path.join(diretorio_entrada, arquivo_zip)
    nome_base = os.path.splitext(arquivo_zip)[0]  # Remove a extensão .zip
    
    # Cria uma pasta temporária para extrair o conteúdo do arquivo .zip
    pasta_temporaria = os.path.join(diretorio_saida, f"temp_{nome_base}")
    os.makedirs(pasta_temporaria, exist_ok=True)
    
    # Extrai o conteúdo do arquivo .zip para a pasta temporária
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(pasta_temporaria)
    
    # Lista todos os arquivos extraídos na pasta temporária
    arquivos_extraidos = [
        os.path.join(pasta_temporaria, f) for f in os.listdir(pasta_temporaria)
    ]
    
    # Itera sobre cada arquivo extraído
    for caminho_arquivo in arquivos_extraidos:
        extensao_arquivo = os.path.splitext(caminho_arquivo)[1]  # Obtém a extensão
        base_arquivo = os.path.splitext(os.path.basename(caminho_arquivo))[0]  # Nome sem extensão
        
        # Cria um novo nome padronizado
        novo_nome = f"{base_arquivo}_{nome_base}{extensao_arquivo}"
        novo_caminho = os.path.join(diretorio_saida, novo_nome)
        
        # Renomeia e move o arquivo para o diretório de saída
        os.rename(caminho_arquivo, novo_caminho)
    
    # Remove a pasta temporária após processar todos os arquivos
    arcpy.AddMessage(f"Processando {arquivo_zip} concluído.")
    os.rmdir(pasta_temporaria)  # Remove a pasta temporária

arcpy.AddMessage("Todos os arquivos foram descompactados e renomeados com sucesso!")

#### Merge CAR

In [ ]:
# Feature class resultante do merge
merged_CARs_UF_name = "CARinicial"

shapefiles = []

# Loop pelos shapefiles no diretório
for file in os.listdir(diretorio_saida):
    if file.endswith(".shp"):
        shapefile_path = os.path.join(diretorio_saida, file)
        shapefiles.append(shapefile_path)
        print(f"Adicionado ao merge: {shapefile_path}")

# Merge
arcpy.Merge_management(shapefiles, os.path.join(GDB, merged_CARs_UF_name))
print(f"Merge concluído! Resultado salvo como {merged_CARs_UF_name} no GDB.")

In [ ]:
CAR = "CARinicial"

#### Exportar Tabela CAR

In [ ]:
TAB = "Tabela_Rel_SICAR"

arcpy.conversion.ExportTable(
    CAR,
    f"{GDB}\\{TAB}")

#### Limpar CAR

In [ ]:
fields = arcpy.ListFields(CAR)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.DeleteField_management(CAR, ["cod_tema", "nom_tema", "mod_fiscal", "num_area", "ind_status", "ind_tipo", "des_condic", "municipio", "cod_estado", "dat_criaca", "dat_atuali"])

### Corrigir Geometrias

#### Repair Geometry

In [ ]:
arcpy.management.RepairGeometry(CAR,
                                delete_null       = "DELETE_NULL",
                                validation_method = "ESRI")

### Reprojetar e Área Total do CAR

#### **Projeção recomendada pelo IBGE**

A projeção Cônica Equivalente de Albers com o datum horizontal SIRGAS2000 é recomendada pelo Instituto Brasileiro de Geografia e Estatística (IBGE) para preservação e cálculo de áreas no território nacional.

O IBGE utiliza para o cálculo de área a Projeção Equivalente de Albers, que tem
como característica principal a equivalência em área. Essa projeção é formada por um
cone ao redor do globo, sob o qual as feições da superfície terrestre são projetadas. O
cone intercepta o globo em dois paralelos padrão, com os meridianos formando linhas
retas com a origem em um ponto central na geratriz do cone, enquanto os paralelos
formam círculos concêntricos em torno deste ponto.

Os parâmetros da projeção adotada são:

* Meridiano Central -54º
* Latitude de Origem -12º
* 1º Paralelo Padrão -2º
* 2º Paralelo Padrão -22º
* Origem E: 5.000.000
* Origem N: 10.000.000
* Área de abrangência:
    - canto inferior esquerdo (E, N): 2.800.000, 7.350.000
    - canto superior direito (E, N): 8.210.000, 12.200.000

O Datum horizontal adotado é o **SIRGAS2000**.

Infelizmente a Projeção de Albers com os parâmetros do IBGE não possuem um
código EPSG (sigla para “European Petroleum Survey Group”) definido que possa ser
facilmente encontrado e utilizado nos softwares de geoprocessamento.

*Referência: [Informações técnicas e legais para a utilização dos dados publicados - IBGE, 2023](biblioteca.ibge.gov.br/index.php/biblioteca-catalogo?view=detalhes&id=2101998). 
Acesso em novembro de 2024.*

**Formato WKT:**

In [ ]:
PROJCS["Conica_Equivalente_de_Albers_Brasil",
 GEOGCS["GCS_SIRGAS2000",
 DATUM["D_SIRGAS2000",
 SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
 PRIMEM["Greenwich",0],
 UNIT["Degree",0.017453292519943295]],
 PROJECTION["Albers"],
 PARAMETER["standard_parallel_1",-2],
 PARAMETER["standard_parallel_2",-22],
 PARAMETER["latitude_of_origin",-12],
 PARAMETER["central_meridian",-54],
 PARAMETER["false_easting",5000000],
 PARAMETER["false_northing",10000000],
 UNIT["Meter",1]]

**Formato Proj4:**

In [ ]:
"+proj=aea +lat_0=-12 +lon_0=-54 +lat_1=-2 +lat_2=-22 +x_0=5000000 +y_0=10000000+ellps=GRS80 +units=m +no_defs"

#### **Criar Sistema de coordenadas personalizado**

In [ ]:
albers_brasil = arcpy.SpatialReference()
albers_brasil.loadFromString("""
PROJCS["Conica_Equivalente_de_Albers_Brasil",
    GEOGCS["GCS_SIRGAS2000",
        DATUM["D_SIRGAS2000",
            SPHEROID["Geodetic_Reference_System_of_1980",6378137,298.2572221009113]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295]],
    PROJECTION["Albers"],
    PARAMETER["standard_parallel_1",-2],
    PARAMETER["standard_parallel_2",-22],
    PARAMETER["latitude_of_origin",-12],
    PARAMETER["central_meridian",-54],
    PARAMETER["false_easting",5000000],
    PARAMETER["false_northing",10000000],
    UNIT["Meter",1]]
""")

In [ ]:
albers_brasil = arcpy.Describe(MUN).spatialReference

#### Projetar

In [ ]:
arcpy.management.Project(CAR, 
                         "CAR_prj", 
                         albers_brasil
                        )

In [ ]:
CAR = "CAR_prj"

### Área total do imóvel CAR

#### <mark>Onde Estamos</mark>?

In [ ]:
CAR = "CAR_prj"

#### Adicionar nova coluna

In [ ]:
arcpy.management.AddField(CAR,
                          "AREAha",
                          "DOUBLE"
                         )

#### Calcular geometria : área em hectares

In [ ]:
arcpy.management.CalculateField(CAR,
                                "AREAha", 
                                "round(!Shape.area@HECTARES!,4)", 
                                "PYTHON")

## Dados Municípios

#### <mark>Onde Estamos</mark>?

CAR = "CAR_prj"

#### **Importar Municípios**

Referência: [Informações técnicas e legais para a utilização dos dados publicados (IBGE, 2023)](https://biblioteca.ibge.gov.br/index.php/biblioteca-catalogo?view=detalhes&id=2101998). Acesso em novembro de 2024.

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(r"D:\CAR_2025\CAR_R\dados_limpos\municipios_unzipped\municipios_prj.shp",
                                            GDB,
                                            "MUN")

#### **Colunas e Estrutura do Vetor Municipios**

In [ ]:
MUN = "MUN"

fields = arcpy.ListFields(MUN)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

#### **Recuperar Código Municípios de acordo com IBGE**

In [ ]:
arcpy.AddField_management(CAR, 
                          "CD_MUN", 
                          "TEXT", 
                          field_length=7)

In [ ]:
with arcpy.da.UpdateCursor(CAR, ["cod_imovel", "CD_MUN"]) as cursor:
    for row in cursor:

        cod_imovel_value = row[0]
        cd_mun_value = cod_imovel_value[3:10]  # pega os 7 caracteres a partir da posição 4
        
        row[1] = cd_mun_value
        cursor.updateRow(row) 

print("Campo 'CD_MUN' preenchido com sucesso!")

In [ ]:
fields = arcpy.ListFields(CAR)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

#### **Join de Municipios para base CAR**

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR,                    # Tabela de entrada (CAR)
    in_field   = "CD_MUN",               # Campo em CAR para a junção
    join_table = MUN,                    # Tabela de junção (MUN)
    join_field = "CD_MUN",               # Campo em MUN para a junção
    fields     = ["NM_MUN", "SIGLA_UF"]  # Campos de MUN a serem adicionados em CAR
)

#### Exportar Tabela CAR

In [ ]:
TAB = "Tabela_Rel_MUN"

arcpy.conversion.ExportTable(
    CAR,
    f"{GDB}\\{TAB}")

#### **Limpar CAR**

In [ ]:
arcpy.DeleteField_management(CAR, ["NM_MUN", "SIGLA_UF"])

# Processamento 

## Amazonia Legal?

#### <mark>Onde Estamos</mark>?

In [ ]:
CAR = "CAR_prj" # CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha)

#### Importar AL

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(r"D:\CAR_2025\CAR_R\dados_limpos\amazonia_legal_unzipped\amazonia_legal_prj.shp",
                                            GDB,
                                            "AL")

#### **Colunas e Estrutura do Vetor Amazônia Legal**

In [ ]:
AL = "AL"

fields = arcpy.ListFields(AL)
for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.AddField(CAR,
                          "AL",
                          "SHORT"
                         )

#### Select By Location

In [ ]:
arcpy.management.SelectLayerByLocation(
    in_layer        = CAR,               # Camada onde as feições serão selecionadas
    overlap_type    = "INTERSECT",    
    select_features = AL,                # Camada usada para a seleção
    selection_type  = "NEW_SELECTION")

Camadas que intersectam os limites da Amazônia Legal serão preenchidas com **1**

In [ ]:
arcpy.management.CalculateField(
    CAR,
    "AL",
    expression      = 1,             
    expression_type = "PYTHON3"
)

In [ ]:
arcpy.management.SelectLayerByLocation(
    in_layer        = CAR,              
    overlap_type    = "INTERSECT",    
    select_features = AL,                
    selection_type  = "SWITCH_SELECTION")

As demais serão preenchidas com **0**

In [ ]:
arcpy.management.CalculateField(
    CAR,
    "AL",
    expression      = 0,             
    expression_type = "PYTHON3"
)

#### Limpar a seleção

In [ ]:
arcpy.management.SelectLayerByAttribute(CAR, "CLEAR_SELECTION")

#### Exportar Tabela CAR

In [ ]:
TAB = "Tabela_Rel_AL_AreaCAR"

arcpy.conversion.ExportTable(
    CAR,
    f"{GDB}\\{TAB}")

## Qual o Bioma?

#### <mark>Onde Estamos</mark>?

In [ ]:
CAR = "CAR_prj" # CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL

#### **Importar Biomas**

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(r"D:\CAR_2025\CAR_R\dados_limpos\biomas_unzipped\bioma_prj.shp",  # resultado Etapa 1
                                            GDB,
                                            "BIOMA")

#### **Colunas e Estrutura do Vetor Bioma**

In [ ]:
BIOMA = "BIOMA"

fields = arcpy.ListFields(BIOMA)
for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")
    
fields = arcpy.ListFields(CAR)
for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

#### Spatial Join

In [ ]:
field_mappings = arcpy.FieldMappings()

# Adiciona o campo "cod_imovel" de CAR
cod_imovel_field_map = arcpy.FieldMap()
cod_imovel_field_map.addInputField(CAR, "cod_imovel")
field_mappings.addFieldMap(cod_imovel_field_map)

# Adiciona o campo "Bioma" de BIOMA
bioma_field_map = arcpy.FieldMap()
bioma_field_map.addInputField(BIOMA, "Bioma")
field_mappings.addFieldMap(bioma_field_map)

In [ ]:
arcpy.analysis.SpatialJoin(
    target_features   = CAR, 
    join_features     = BIOMA, 
    out_feature_class = "CAR_BIOMA_spatial_join", 
    join_operation    = "JOIN_ONE_TO_ONE", 
    join_type         = "KEEP_ALL", 
    field_mapping     = field_mappings, 
    match_option      = "INTERSECT")

#### Join

In [ ]:
CAR = "CAR_prj" # CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL
CAR_BIOMA_spatial_join = "CAR_BIOMA_spatial_join"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR, 
    in_field   = "cod_imovel", 
    join_table = CAR_BIOMA_spatial_join, 
    join_field = "cod_imovel", 
    fields     = ["Bioma"])

## Reserva Legal Requerida: Artigo 12 do Código Florestal

#### <mark>Onde Estamos</mark>?

In [ ]:
CAR = "CAR_prj" # CAR merged, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma

Os percentuais de Reserva Legal (RL) na Amazônia Legal não dependem apenas dos limites do bioma Amazônia ou do bioma Cerrado, mas sim das fitofisionomias presentes no imóvel. 

O **Art. 12 do Código Florestal** determina que os percentuais mínimos de RL na Amazônia Legal são:
 
* **80%** para áreas de **florestas**.
* **35%** para áreas de **cerrado**.
* **20%** para áreas de **campos gerais**.

Para imóveis com mais de uma fitofisionomia, os percentuais devem ser aplicados **separadamente** para cada formação.

#### **Amazônia Legal? Sim**

In [ ]:
sql_AL = "AL = 1"

arcpy.conversion.ExportFeatures(CAR, 
                                f"{GDB}/CAR_AL", 
                                sql_AL, 
                                "NOT_USE_ALIAS")

#### **Amazônia Legal? Não**

In [ ]:
sql_nAL = "AL <> 1 OR AL IS NULL"

arcpy.conversion.ExportFeatures(CAR, 
                                f"{GDB}/CAR_nAL", 
                                sql_nAL, 
                                "NOT_USE_ALIAS")

### Reserva Legal Requerida - Fora da Amazônia Legal

#### <mark>Onde Paramos?</mark> 

In [ ]:
# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR     = "CAR_prj" 

# Imóveis na Amazônia Legal e fora da Amazônia Legal serão tratados separadamente a partir daqui.
# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR_AL  = "CAR_AL"  # in  AL
CAR_nAL = "CAR_nAL" # out AL

Art. 12. Todo imóvel rural deve manter área com cobertura de vegetação nativa, a título de Reserva Legal, sem prejuízo da aplicação das normas sobre as Áreas de Preservação Permanente, observados os seguintes percentuais mínimos em relação à área do imóvel*, excetuados os casos previstos no art. 68 desta LeiLei nº 12.727, de 2012)*:

I - localizado na Amazônia Legal:

a. **80%** no imóvel situado em área de **florestas**.

b. **35%** no imóvel situado em área de **cerrado**.

c. **20%** no imóvel situado em área de **campos gerais**.

<mark>II - localizado nas demais regiões do País: **20%**.</mark>

§ 2º O percentual de Reserva Legal em imóvel situado em área de formações florestais, de cerrado ou de campos gerais na Amazônia Legal será definido considerando separadamente os índices contidos nas alíneas a, b e c do inciso I do caput .

#### Percentual Requerido = 20%

In [ ]:
arcpy.management.AddField(CAR_nAL,
                          "ARL",
                          "DOUBLE"
                         )

In [ ]:
arcpy.management.CalculateField(CAR_nAL,
                                "ARL", 
                                "0.2", 
                                "PYTHON3")

In [ ]:
arcpy.management.AddField(CAR_nAL,
                          "RLha",
                          "DOUBLE"
                         )

In [ ]:
arcpy.management.CalculateField(CAR_nAL,
                                "RLha", 
                                "round(!AREAha!*!ARL!,4)", 
                                "PYTHON3")

### Dados Fitofisionomia - CAR na Amazônia Legal 

#### <mark>Onde Paramos?</mark> 

In [ ]:
# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR     = "CAR_prj" 

# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR_nAL = "CAR_nAL" # out AL / com ARL, RL
CAR_AL  = "CAR_AL"  # in  AL 

#### **Importar Fitofisionomia simplificado**

In [ ]:
arcpy.FeatureClassToFeatureClass_conversion(r"D:\CAR_2025\CAR_R\dados_limpos\fitofisionomia_unzipped\fitofisionomia_prj_final.shp",
                                            GDB,
                                            "FITO")

#### Colunas e Estrutura do Vetor Fitofisionomia Simplificado*

In [ ]:
FITO = "FITO"

fields = arcpy.ListFields(FITO)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

#### Intersection

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR, BIOMA], 
    out_feature_class = "CAR_AL_FITO")

#### **Dissolve**

In [ ]:
CAR_AL_FITO = "CAR_AL_FITO"

In [ ]:
arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_AL_FITO, 
    out_feature_class = "CAR_AL_FITO_dissolve", 
    dissolve_field    = ["cod_imovel", "legenda_1", "Bioma"],  
    multi_part        = "MULTI_PART" #"SINGLE_PART" 
)

### Reserva Legal Requerida - Amazônia Legal

#### <mark>Onde Paramos?</mark>

In [ ]:
# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR     = "CAR_prj" 

# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR_nAL = "CAR_nAL" # out AL / com ARL, RL

CAR_AL      = "CAR_AL"  # in  AL 
CAR_AL_FITO = "CAR_AL_FITO_dissolve" # cod_imovel + legenda_1 + bioma

Os percentuais de Reserva Legal (RL) na Amazônia Legal não dependem apenas dos limites do bioma Amazônia ou do bioma Cerrado, mas sim das fitofisionomias presentes no imóvel. 

O **Art. 12 do Código Florestal** determina que os percentuais mínimos de RL na Amazônia Legal são:
 
* **80%** no imóvel situado em área de **florestas**.
* **35%** no imóvel situado em área de **cerrado**.
* **20%** no imóvel situado em área de **campos gerais**.

Para imóveis com mais de uma fitofisionomia, os percentuais devem ser aplicados **separadamente** para cada formação.

#### Percentual Requerido

In [ ]:
arcpy.management.AddField(CAR_AL_FITO,
                          "ARL",
                          "DOUBLE"
                         )

In [ ]:
code_block = """
def calcular(legenda_1, Bioma):
    if legenda_1 == "floresta":
        return 0.80
    elif legenda_1 == "savana":
        return 0.35
    elif legenda_1 == "campinarana":
        return 0.20
    elif legenda_1 == "outro":
        if Bioma == "Amazônia":
            return 0.80
        elif Bioma in ["Cerrado", "Pantanal"]:
            return 0.35
    return None  # Retorna None se nenhuma condição for atendida
"""

In [ ]:
arcpy.management.CalculateField(
    in_table        = CAR_AL_FITO,
    field           = "ARL",
    expression      = "calcular(!legenda_1!, !Bioma!)",
    expression_type = "PYTHON3",
    code_block      = code_block
)

#### Area Fitofisionomia

In [ ]:
arcpy.management.AddField(CAR_AL_FITO,
                          "AREAfitoha",
                          "DOUBLE"
                         )

In [ ]:
arcpy.management.CalculateField(CAR_AL_FITO,
                                "AREAfitoha", 
                                "round(!Shape.area@HECTARES!,4)", 
                                "PYTHON")

#### Área de Reserva Legal Requerida por Fitofisionomia e por Imóvel na Amazônia Legal

In [ ]:
arcpy.management.AddField(CAR_AL_FITO,
                          "RLha",
                          "DOUBLE"
                         )

In [ ]:
arcpy.management.CalculateField(CAR_AL_FITO,
                                "RLha", 
                                "round(!AREAfitoha!*!ARL!,4)", 
                                "PYTHON")

#### Identificador Individual

In [ ]:
arcpy.management.AddField(CAR_AL_FITO, "ID_UNICO", "TEXT")

In [ ]:
with arcpy.da.UpdateCursor(CAR_AL_FITO, ["ID_UNICO"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"ID_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

## Quantificação Vegetação Nativa e EVN 

### MapBiomas x CAR: na Amazônia Legal

###### <mark>Onde Estamos?</mark>

In [ ]:
# CAR merged, limpo, com CD_MUN, geometria corrigida, reprojetado, área total (ha), AL, Bioma
CAR     = "CAR_prj" 

CAR_AL  = "CAR_AL"  # in  AL 
CAR_AL_FITO = "CAR_AL_FITO_dissolve" # cod_imovel + legenda_1 + bioma / com ARL, RL

MBp = "brasil_coverage_23_reclass_prj_POL"

#### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL_FITO,
    zone_fields       = "ID_UNICO",
    in_class_features = MBp,
    out_table         = "Tabulate_Inter_CAR_AL_MBp",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_AL_MBp  = "Tabulate_Inter_CAR_AL_MBp"

#### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_AL_MBp, 
    fields      = "ID_UNICO", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_AL_MBp_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_AL_MBp  = "Tabulate_Inter_CAR_AL_MBp_pivot"

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_AL_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_AL_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Join

In [ ]:
Tabulate_Inter_CAR_AL_MBp  = "Tabulate_Inter_CAR_AL_MBp_pivot"
CAR_AL_FITO = "CAR_AL_FITO_dissolve"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL_FITO, 
    in_field   = "ID_UNICO", 
    join_table = Tabulate_Inter_CAR_AL_MBp,
    join_field = "ID_UNICO", 
    fields     = ["gridcode1", "gridcode2", "gridcode3", "gridcode4", "gridcode5", "gridcode6"])

#### Dissolve

In [ ]:
statistics_fields = [
    ["AREAfitoha", "SUM"],
    ["RLha"      , "SUM"],
    ["gridcode1" , "SUM"],
    ["gridcode2" , "SUM"],
    ["gridcode3" , "SUM"],
    ["gridcode4" , "SUM"],
    ["gridcode5" , "SUM"],
    ["gridcode6" , "SUM"],
    ["legenda_1" ,  "CONCATENATE"]]

In [ ]:
arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_AL_FITO, 
    out_feature_class = "CAR_AL_FITO_MB", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = ";"
)

#### Ajustando Colunas: MapBiomas

In [ ]:
CAR_AL_FITO_MB = "CAR_AL_FITO_MB"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_AL_FITO_MB,
                          "FITO",
                          "TEXT")

#---------------------------------
arcpy.management.AddField(CAR_AL_FITO_MB,
                          "FITOha",
                          "DOUBLE")

#---------------------------------
arcpy.management.AddField(CAR_AL_FITO_MB,
                          "RLha",
                          "DOUBLE")

#---------------------------------
arcpy.management.AddField(CAR_AL_FITO_MB,
                          "VNha",
                          "DOUBLE")

arcpy.management.AddField(CAR_AL_FITO_MB,
                          "SOJAha",
                          "DOUBLE")

arcpy.management.AddField(CAR_AL_FITO_MB,
                          "PASTOha",
                          "DOUBLE")

arcpy.management.AddField(CAR_AL_FITO_MB,
                          "C_AGUAha",
                          "DOUBLE")

arcpy.management.AddField(CAR_AL_FITO_MB,
                          "OUTha",
                          "DOUBLE")

arcpy.management.AddField(CAR_AL_FITO_MB,
                          "outrAGROha",
                          "DOUBLE")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "FITOha", 
                                "round(!SUM_AREAfitoha!,4)", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "RLha", 
                                "round(!SUM_RLha!,4)", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "VNha", 
                                "round(!SUM_gridcode1!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "C_AGUAha", 
                                "round(!SUM_gridcode2!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "SOJAha", 
                                "round(!SUM_gridcode3!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "PASTOha", 
                                "round(!SUM_gridcode4!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "outrAGROha", 
                                "round(!SUM_gridcode5!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(CAR_AL_FITO_MB,
                                "OUTha", 
                                "round(!SUM_gridcode6!,4)", 
                                "PYTHON3")

#### Ajustando Colunas: Fitofisionomia

In [ ]:
arcpy.management.CalculateField(
    in_table        = CAR_AL_FITO_MB,
    field           = "FITO",
    expression      = "clean_value(!CONCATENATE_legenda_1!)",
    expression_type = "PYTHON3",
    code_block="""
def clean_value(value):
    if not value:
        return ""
    items = value.split(";")
    unique_items = list(dict.fromkeys(items))
    clean_text = ";".join(unique_items)
    return clean_text[1:] if clean_text.startswith(";") else clean_text
"""
)

In [ ]:
fieldObs = arcpy.ListFields(CAR_AL_FITO_MB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_AL_FITO_MB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Join Final

In [ ]:
CAR_AL_FITO_MB = "CAR_AL_FITO_MB"
CAR_AL = "CAR_AL" 

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = CAR_AL_FITO_MB,
    join_field = "cod_imovel", 
    fields     = ["FITO", "FITOha", "RLha", "VNha", "SOJAha", "PASTOha", "C_AGUAha", "OUTha", "outrAGROha"])

### MapBiomas x CAR: Fora da Amazônia Legal

#### Cerrado

###### <mark>Onde Estamos?</mark>

In [ ]:
CAR_nAL = "CAR_nAL"
MBp = "brasil_coverage_23_reclass_prj_POL"

###### Filtro

In [ ]:
sql_nALcerr = '"Bioma" = \'Cerrado\''
sql_nALcerr = "Bioma = 'Cerrado'"


arcpy.conversion.ExportFeatures(CAR_nAL, 
                                f"{GDB}/CAR_nALcerrado", 
                                sql_nALcerr, 
                                "NOT_USE_ALIAS")

In [ ]:
CAR_nALcerrado = "CAR_nALcerrado"

###### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = MBp,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_MBp", # Tabulate_Inter_CAR_nALcerrado_MBp
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_MBp = "Tabulate_Inter_CAR_nALcerrado_MBp"

###### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcerrado_MBp, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcerrado_MBp_pivot")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_MBp = "Tabulate_Inter_CAR_nALcerrado_MBp_pivot"

###### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcerrado_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcerrado_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

###### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_nALcerrado_MBp = "Tabulate_Inter_CAR_nALcerrado_MBp_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "VNha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "SOJAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "PASTOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "C_AGUAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "OUTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_MBp,
                          "outrAGROha",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "VNha", 
                                "round(!gridcode1!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "C_AGUAha", 
                                "round(!gridcode2!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "SOJAha", 
                                "round(!gridcode3!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "PASTOha", 
                                "round(!gridcode4!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "outrAGROha", 
                                "round(!gridcode5!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_MBp,
                                "OUTha", 
                                "round(!gridcode6!,4)", 
                                "PYTHON3")

###### Join Final

In [ ]:
CAR_nALcerrado = "CAR_nALcerrado" # out AL / com ARL, RL

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_MBp,
    join_field = "cod_imovel", 
    fields     = ["VNha", "SOJAha", "PASTOha", "C_AGUAha", "OUTha", "outrAGROha"])

####  Mata Atlântica

###### <mark>Onde Estamos?</mark>

In [ ]:
CAR_nAL = "CAR_nAL"
MBp = "brasil_coverage_23_reclass_prj_POL"

###### Filtro

In [ ]:
sql_nALmata = "Bioma = 'Mata Atlântica'"


arcpy.conversion.ExportFeatures(CAR_nAL, 
                                f"{GDB}/CAR_nALmata", 
                                sql_nALmata, 
                                "NOT_USE_ALIAS")

In [ ]:
CAR_nALmata = "CAR_nALmata"

###### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = MBp,
    out_table         = "Tabulate_Inter_CAR_nALmata_MBp", # Tabulate_Inter_CAR_nALcerrado_MBp
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_nALmata_MBp = "Tabulate_Inter_CAR_nALmata_MBp"

###### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALmata_MBp, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALmata_MBp_pivot")

In [ ]:
Tabulate_Inter_CAR_nALmata_MBp = "Tabulate_Inter_CAR_nALmata_MBp_pivot"

###### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALmata_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALmata_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

###### Ajuste de colunas

In [ ]:
Tabulate_Inter_CAR_nALmata_MBp = "Tabulate_Inter_CAR_nALmata_MBp_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "VNha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "SOJAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "PASTOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "C_AGUAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "OUTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_MBp,
                          "outrAGROha",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "VNha", 
                                "round(!gridcode1!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "C_AGUAha", 
                                "round(!gridcode2!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "SOJAha", 
                                "round(!gridcode3!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "PASTOha", 
                                "round(!gridcode4!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "outrAGROha", 
                                "round(!gridcode5!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_MBp,
                                "OUTha", 
                                "round(!gridcode6!,4)", 
                                "PYTHON3")

###### Join Final

In [ ]:
CAR_nALmata = "CAR_nALmata" # out AL / com ARL, RL

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_MBp,
    join_field = "cod_imovel", 
    fields     = ["VNha", "SOJAha", "PASTOha", "C_AGUAha", "OUTha", "outrAGROha"])

#### Caatinga

###### <mark>Onde Estamos?</mark>

In [ ]:
CAR_nAL = "CAR_nAL"
MBp = "brasil_coverage_23_reclass_prj_POL"

###### Filtro

In [ ]:
sql_nALcaatinga = "Bioma = 'Caatinga'"


arcpy.conversion.ExportFeatures(CAR_nAL, 
                                f"{GDB}/CAR_nALcaatinga", 
                                sql_nALcaatinga, 
                                "NOT_USE_ALIAS")

In [ ]:
CAR_nALcaatinga = "CAR_nALcaatinga"

###### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = MBp,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_MBp", # Tabulate_Inter_CAR_nALcerrado_MBp
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_MBp = "Tabulate_Inter_CAR_nALcaatinga_MBp"

###### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcaatinga_MBp, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcaatinga_MBp_pivot")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_MBp = "Tabulate_Inter_CAR_nALcaatinga_MBp_pivot"

###### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcaatinga_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcaatinga_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

###### Ajuste Colunas

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_MBp = "Tabulate_Inter_CAR_nALcaatinga_MBp_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "VNha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "SOJAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "PASTOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "C_AGUAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "OUTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                          "outrAGROha",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "VNha", 
                                "round(!gridcode1!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "C_AGUAha", 
                                "round(!gridcode2!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "SOJAha", 
                                "round(!gridcode3!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "PASTOha", 
                                "round(!gridcode4!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "outrAGROha", 
                                "round(!gridcode5!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_MBp,
                                "OUTha", 
                                "round(!gridcode6!,4)", 
                                "PYTHON3")

###### Join Final

In [ ]:
CAR_nALcaatinga = "CAR_nALcaatinga" # out AL / com ARL, RL

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_MBp,
    join_field = "cod_imovel", 
    fields     = ["VNha", "SOJAha", "PASTOha", "C_AGUAha", "OUTha", "outrAGROha"])

#### Pantanal, Pampa

###### <mark>Onde Estamos?</mark>

In [ ]:
CAR_nAL = "CAR_nAL"
MBp = "brasil_coverage_23_reclass_prj_POL"

###### Filtro

In [ ]:
sql_nALPamPan = "Bioma = 'Pantanal' Or Bioma = 'Pampa'"


arcpy.conversion.ExportFeatures(CAR_nAL, 
                                f"{GDB}/CAR_nALPamPan", 
                                sql_nALPamPan, 
                                "NOT_USE_ALIAS")

In [ ]:
CAR_nALPamPan = "CAR_nALPamPan"

###### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = MBp,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_MBp", # Tabulate_Inter_CAR_nALcerrado_MBp
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_MBp = "Tabulate_Inter_CAR_nALPamPan_MBp"

###### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALPamPan_MBp, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALPamPan_MBp_pivot")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_MBp = "Tabulate_Inter_CAR_nALPamPan_MBp_pivot"

###### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALPamPan_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALPamPan_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

###### Ajuste Colunas

In [ ]:
Tabulate_Inter_CAR_nALPamPan_MBp = "Tabulate_Inter_CAR_nALPamPan_MBp_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "VNha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "SOJAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "PASTOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "C_AGUAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "OUTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_MBp,
                          "outrAGROha",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "VNha", 
                                "round(!gridcode1!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "C_AGUAha", 
                                "round(!gridcode2!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "SOJAha", 
                                "round(!gridcode3!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "PASTOha", 
                                "round(!gridcode4!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "outrAGROha", 
                                "round(!gridcode5!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_MBp,
                                "OUTha", 
                                "round(!gridcode6!,4)", 
                                "PYTHON3")

###### Join Final

In [ ]:
CAR_nALPamPan = "CAR_nALPamPan" # out AL / com ARL, RL

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_MBp,
    join_field = "cod_imovel", 
    fields     = ["VNha", "SOJAha", "PASTOha", "C_AGUAha", "OUTha", "outrAGROha"])

### EVN

#### <mark>Onde Estamos?</mark>

In [ ]:
CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

#### EVN: na Amazônia Legal

In [ ]:
CAR_AL = "CAR_AL"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_AL,
                          "EVNha",
                          "DOUBLE")

In [ ]:
fieldObs = arcpy.ListFields(CAR_AL)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_AL, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_AL,
                                "EVNha", 
                                "round(!VNha! - !RLha!,4)", 
                                "PYTHON3")

#### EVN: Fora da Amazônia Legal

##### Cerrado

In [ ]:
CAR_nALcerrado = "CAR_nALcerrado"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_nALcerrado,
                          "EVNha",
                          "DOUBLE")

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcerrado)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcerrado, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_nALcerrado,
                                "EVNha", 
                                "round(!VNha! - !RLha!,4)", 
                                "PYTHON3")

##### Mata Atlântica

In [ ]:
CAR_nALmata = "CAR_nALmata"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_nALmata,
                          "EVNha",
                          "DOUBLE")

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALmata)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALmata, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_nALmata,
                                "EVNha", 
                                "round(!VNha! - !RLha!,4)", 
                                "PYTHON3")

##### Caatinga

In [ ]:
CAR_nALcaatinga = "CAR_nALcaatinga"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_nALcaatinga,
                          "EVNha",
                          "DOUBLE")

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcaatinga)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcaatinga, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_nALcaatinga,
                                "EVNha", 
                                "round(!VNha! - !RLha!,4)", 
                                "PYTHON3")

##### Pantanal, Pampa

In [ ]:
CAR_nALmata = "CAR_nALPamPan"

In [ ]:
#---------------------------------
arcpy.management.AddField(CAR_nALPamPan,
                          "EVNha",
                          "DOUBLE")

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALPamPan)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALPamPan, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
#---------------------------------
arcpy.management.CalculateField(CAR_nALPamPan,
                                "EVNha", 
                                "round(!VNha! - !RLha!,4)", 
                                "PYTHON3")

##  Desmatamento PRODES

### <mark>Onde Estamos?</mark>

In [ ]:
PRODES          = "prodes_final"

CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

### Desmatamento: na Amazônia Legal

#### Tabulate Intersection

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = PRODES,
    out_table         = "Tabulate_Inter_CAR_AL_PRODES",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_AL_PRODES  = "Tabulate_Inter_CAR_AL_PRODES"

#### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_AL_PRODES, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_AL_MBp_pivot")

In [ ]:
Tabulate_Inter_CAR_AL_PRODES  = "Tabulate_Inter_CAR_AL_PRODES_pivot"

#### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_AL_MBp)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_AL_MBp, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_AL_PRODES  = "Tabulate_Inter_CAR_AL_PRODES_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P08_19",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P2020",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P2021",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P2022",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P2023",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_PRODES,
                          "P2024",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P08_19", 
                                "round(!gridcode2008! + !gridcode2009! + !gridcode2010! + !gridcode2011! + !gridcode2012! + !gridcode2013! + !gridcode2014! + !gridcode2015! + !gridcode2016! + !gridcode2017! + !gridcode2018!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P2020", 
                                "round(!gridcode2020!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P2021", 
                                "round(!gridcode2021!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P2022", 
                                "round(!gridcode2022!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P2023", 
                                "round(!gridcode2023!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_PRODES,
                                "P2024", 
                                "round(!gridcode2024!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
CAR_AL = "CAR_AL"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL , 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_PRODES,
    join_field = "cod_imovel", 
    fields     = ["P08_19", "P2020", "P2021", "P2022", "P2023", "P2024"])

### Desmatamento: Fora da Amazônia Legal

#### Cerrado

##### Tabulate Intersection

In [ ]:
CAR_nALcerrado = "CAR_nALcerrado"

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = PRODES,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_PRODES",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_PRODES  = "Tabulate_Inter_CAR_nALcerrado_PRODES"

##### Pivot Table

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcerrado_PRODES, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcerrado_PRODES_pivot")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_PRODES  = "Tabulate_Inter_CAR_nALcerrado_PRODES_pivot"

##### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcerrado_PRODES)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcerrado_PRODES, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

##### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_nALcerrado_PRODES  = "Tabulate_Inter_CAR_nALcerrado_PRODES_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P08_19",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P2020",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P2021",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P2022",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P2023",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                          "P2024",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P08_19", 
                                "round(!gridcode2008! + !gridcode2009! + !gridcode2010! + !gridcode2011! + !gridcode2012! + !gridcode2013! + !gridcode2014! + !gridcode2015! + !gridcode2016! + !gridcode2017! + !gridcode2018!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P2020", 
                                "round(!gridcode2020!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P2021", 
                                "round(!gridcode2021!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P2022", 
                                "round(!gridcode2022!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P2023", 
                                "round(!gridcode2023!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_PRODES,
                                "P2024", 
                                "round(!gridcode2024!,4)", 
                                "PYTHON3")

##### Join

In [ ]:
CAR_nALcerrado = "CAR_nALcerrado"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_PRODES,
    join_field = "cod_imovel", 
    fields     = ["P08_19", "P2020", "P2021", "P2022", "P2023", "P2024"])

#### Mata Atlântica

##### Tabulate Intersection

In [ ]:
CAR_nALmata = "CAR_nALmata"

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = PRODES,
    out_table         = "Tabulate_Inter_CAR_nALmata_PRODES",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

##### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALmata_PRODES  = "Tabulate_Inter_CAR_nALmata_PRODES"

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALmata_PRODES, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALmata_PRODES_pivot")

##### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALmata_PRODES  = "Tabulate_Inter_CAR_nALmata_PRODES_pivot"

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALmata_PRODES)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALmata_PRODES, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

##### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_nALmata_PRODES  = "Tabulate_Inter_CAR_nALmata_PRODES_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P08_19",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P2020",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P2021",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P2022",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P2023",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_PRODES,
                          "P2024",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_PRODES,
                                "P08_19", 
                                "round(!gridcode2008! + !gridcode2010! + !gridcode2011! + !gridcode2012! + !gridcode2013! + !gridcode2014! + !gridcode2015! + !gridcode2016! + !gridcode2017! + !gridcode2018!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_PRODES,
                                "P2020", 
                                "round(!gridcode2020!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_PRODES,
                                "P2021", 
                                "round(!gridcode2021!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_PRODES,
                                "P2022", 
                                "round(!gridcode2022!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_PRODES,
                                "P2023", 
                                "round(!gridcode2023!,4)", 
                                "PYTHON3")

##### Join

In [ ]:
CAR_nALcerrado = "CAR_nALmata"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_PRODES,
    join_field = "cod_imovel", 
    fields     = ["P08_19", "P2020", "P2021", "P2022", "P2023", "P2024"])

#### Caatinga

##### Tabulate Intersection

In [ ]:
CAR_nALcaatinga = "CAR_nALcaatinga"

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = PRODES,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_PRODES",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

##### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_PRODES  = "Tabulate_Inter_CAR_nALcaatinga_PRODES"

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcaatinga_PRODES, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcaatinga_PRODES_pivot")

##### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_PRODES  = "Tabulate_Inter_CAR_nALcaatinga_PRODES_pivot"

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcaatinga_PRODES)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcaatinga_PRODES, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

##### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_PRODES  = "Tabulate_Inter_CAR_nALcaatinga_PRODES_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P08_19",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P2020",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P2021",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P2022",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P2023",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                          "P2024",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                                "P08_19", 
                                "round(!gridcode2008! + !gridcode2010! + !gridcode2011! + !gridcode2012! + !gridcode2013! + !gridcode2014! + !gridcode2015! + !gridcode2016! + !gridcode2017! + !gridcode2018!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                                "P2020", 
                                "round(!gridcode2020!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                                "P2021", 
                                "round(!gridcode2021!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                                "P2022", 
                                "round(!gridcode2022!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_PRODES,
                                "P2023", 
                                "round(!gridcode2023!,4)", 
                                "PYTHON3")

##### Join

In [ ]:
CAR_nALcaatinga = "CAR_nALcaatinga"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_PRODES,
    join_field = "cod_imovel", 
    fields     = ["P08_19", "P2020", "P2021", "P2022", "P2023", "P2024"])

#### Pampa, Pantanal

##### Tabulate Intersection

In [ ]:
CAR_nALPamPan = "CAR_nALPamPan"

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = PRODES,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_PRODES",
    class_fields      = "gridcode",
    out_units         = "HECTARES")

##### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALPamPan_PRODES  = "Tabulate_Inter_CAR_nALPamPan_PRODES"

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALPamPan_PRODES, 
    fields      = "cod_imovel", 
    pivot_field = "gridcode", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALPamPan_PRODES_pivot")

##### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALPamPan_PRODES  = "Tabulate_Inter_CAR_nALPamPan_PRODES_pivot"

In [ ]:
fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALPamPan_PRODES)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALPamPan_PRODES, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

##### Ajustando Colunas

In [ ]:
Tabulate_Inter_CAR_nALPamPan_PRODES  = "Tabulate_Inter_CAR_nALPamPan_PRODES_pivot"

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P08_19",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P2020",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P2021",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P2022",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P2023",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                          "P2024",
                          "DOUBLE")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                                "P08_19", 
                                "round(!gridcode2008! + !gridcode2010! + !gridcode2011! + !gridcode2013! + !gridcode2014! + !gridcode2016! + !gridcode2017! + !gridcode2018!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                                "P2020", 
                                "round(!gridcode2020!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                                "P2021", 
                                "round(!gridcode2021!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                                "P2022", 
                                "round(!gridcode2022!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_PRODES,
                                "P2023", 
                                "round(!gridcode2023!,4)", 
                                "PYTHON3")

##### Join

In [ ]:
CAR_nALPamPan = "CAR_nALPamPan"

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_PRODES,
    join_field = "cod_imovel", 
    fields     = ["P08_19", "P2020", "P2021", "P2022", "P2023", "P2024"])

##  Desmatamento Alerta Mapbiomas

### <mark>Onde Estamos?</mark>

In [ ]:
ALERTA = "ALERTAprj_2024"

In [ ]:
CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

### Tabular Intersecção

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = ALERTA,
    out_table         = "Tabulate_Inter_CAR_AL_ALERTA",
    class_fields      = "ANODETEC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = ALERTA,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_ALERTA",
    class_fields      = "ANODETEC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = ALERTA,
    out_table         = "Tabulate_Inter_CAR_nALmata_ALERTA",
    class_fields      = "ANODETEC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = ALERTA,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_ALERTA",
    class_fields      = "ANODETEC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = ALERTA,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_ALERTA",
    class_fields      = "ANODETEC",
    out_units         = "HECTARES")

### Ajustar Colunas

#### Adicionar

In [ ]:
Tabulate_Inter_CAR_AL_ALERTA = "Tabulate_Inter_CAR_AL_ALERTA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_AL_ALERTA,
                          "A2024",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_ALERTA = "Tabulate_Inter_CAR_nALcerrado_ALERTA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_ALERTA,
                          "A2024",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALmata_ALERTA = "Tabulate_Inter_CAR_nALmata_ALERTA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_ALERTA,
                          "A2024",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_ALERTA = "Tabulate_Inter_CAR_nALcaatinga_ALERTA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_ALERTA,
                          "A2024",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_ALERTA = "Tabulate_Inter_CAR_nALPamPan_ALERTA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_ALERTA,
                          "A2024",
                          "DOUBLE")

#### Calcular

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_ALERTA,
                                "A2024", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_ALERTA,
                                "A2024", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_ALERTA,
                                "A2024", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_ALERTA,
                                "A2024", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_ALERTA,
                                "A2024", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_ALERTA,
    join_field = "cod_imovel", 
    fields     = "A2024")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_ALERTA,
    join_field = "cod_imovel", 
    fields     = "A2024")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_ALERTA,
    join_field = "cod_imovel", 
    fields     = "A2024")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_ALERTA,
    join_field = "cod_imovel", 
    fields     = "A2024")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_ALERTA,
    join_field = "cod_imovel", 
    fields     = "A2024")

## Sobreposição Terras Públicas

### <mark>Onde Paramos?</mark>

In [ ]:
CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

### UC - CNUC e ICMBIO

CNUC - GOV - DADOS ABERTOS https://dados.gov.br/dados/conjuntos-dados/unidadesdeconservacao

ICMBIO https://www.gov.br/icmbio/pt-br/assuntos/dados_geoespaciais/mapa-tematico-e-dados-geoestatisticos-das-unidades-de-conservacao-federais

In [ ]:
UC = "UC_CNUC_PI"

In [ ]:
crsUC  = arcpy.Describe(UC).spatialReference
crsCAR_AL = arcpy.Describe(CAR_AL).spatialReference
print("CRS UC    :", crsUC.name)
print("CRS CAR_AL:", crsCAR_AL.name)

#### Tabular Intersecção

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = UC,
    out_table         = "Tabulate_Inter_CAR_AL_UC",
    class_fields      = "UC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = UC,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_UC",
    class_fields      = "UC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = UC,
    out_table         = "Tabulate_Inter_CAR_nALmata_UC",
    class_fields      = "UC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = UC,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_UC",
    class_fields      = "UC",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = UC,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_UC",
    class_fields      = "UC",
    out_units         = "HECTARES")

#### Ajustar Colunas

##### Adicionar

In [ ]:
Tabulate_Inter_CAR_AL_UC = "Tabulate_Inter_CAR_AL_UC"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_AL_UC,
                          "UCha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_UC = "Tabulate_Inter_CAR_nALcerrado_UC"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_UC,
                          "UCha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALmata_UC = "Tabulate_Inter_CAR_nALmata_UC"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_UC,
                          "UCha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_UC = "Tabulate_Inter_CAR_nALcaatinga_UC"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_UC,
                          "UCha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_UC = "Tabulate_Inter_CAR_nALPamPan_UC"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_UC,
                          "UCha",
                          "DOUBLE")

##### Calcular

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_UC,
                                "UCha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_UC,
                                "UCha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_UC,
                                "UCha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_UC,
                                "UCha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_UC,
                                "UCha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
Tabulate_Inter_CAR_AL_UC = "Tabulate_Inter_CAR_AL_UC"

arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_UC,
    join_field = "cod_imovel", 
    fields     = "UCha")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_UC = "Tabulate_Inter_CAR_nALcerrado_UC"

arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_UC,
    join_field = "cod_imovel", 
    fields     = "UCha")

In [ ]:
Tabulate_Inter_CAR_nALmata_UC = "Tabulate_Inter_CAR_nALmata_UC"

arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_UC,
    join_field = "cod_imovel", 
    fields     = "UCha")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_UC = "Tabulate_Inter_CAR_nALcaatinga_UC"

arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_UC,
    join_field = "cod_imovel", 
    fields     = "UCha")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_UC = "Tabulate_Inter_CAR_nALPamPan_UC"

arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_UC,
    join_field = "cod_imovel", 
    fields     = "UCha")

### [QUILOMBOS - INCRA](https://acervofundiario.incra.gov.br/acervo/acv.php)

INCRA https://acervofundiario.incra.gov.br/acervo/acv.php

In [ ]:
QUI = "QUI"

#### Tabular Intersecção

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = QUI,
    out_table         = "Tabulate_Inter_CAR_AL_QUI",
    class_fields      = "QUI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = QUI,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_QUI",
    class_fields      = "QUI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = QUI,
    out_table         = "Tabulate_Inter_CAR_nALmata_QUI",
    class_fields      = "QUI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = QUI,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_QUI",
    class_fields      = "QUI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = QUI,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_QUI",
    class_fields      = "QUI",
    out_units         = "HECTARES")

#### Ajustar Colunas

##### Adicionar

In [ ]:
Tabulate_Inter_CAR_AL_QUI = "Tabulate_Inter_CAR_AL_QUI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_AL_QUI,
                          "QUIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_QUI = "Tabulate_Inter_CAR_nALcerrado_QUI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_QUI,
                          "QUIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALmata_QUI = "Tabulate_Inter_CAR_nALmata_QUI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_QUI,
                          "QUIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_QUI = "Tabulate_Inter_CAR_nALcaatinga_QUI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_QUI,
                          "QUIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_QUI = "Tabulate_Inter_CAR_nALPamPan_QUI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_QUI,
                          "QUIha",
                          "DOUBLE")

##### Calcular

In [ ]:
Tabulate_Inter_CAR_AL_QUI = "Tabulate_Inter_CAR_AL_QUI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_QUI,
                                "QUIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_QUI = "Tabulate_Inter_CAR_nALcerrado_QUI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_QUI,
                                "QUIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALmata_QUI = "Tabulate_Inter_CAR_nALmata_QUI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_QUI,
                                "QUIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_QUI = "Tabulate_Inter_CAR_nALcaatinga_QUI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_QUI,
                                "QUIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_QUI = "Tabulate_Inter_CAR_nALPamPan_QUI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_QUI,
                                "QUIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_QUI,
    join_field = "cod_imovel", 
    fields     = "QUIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_QUI,
    join_field = "cod_imovel", 
    fields     = "QUIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_QUI,
    join_field = "cod_imovel", 
    fields     = "QUIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_QUI,
    join_field = "cod_imovel", 
    fields     = "QUIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_QUI,
    join_field = "cod_imovel", 
    fields     = "QUIha")

### [Glebas Públicas - CNFP](https://www.gov.br/florestal/pt-br/assuntos/cadastro-nacional-de-florestas-publicas)

CNFP https://www.gov.br/florestal/pt-br/assuntos/cadastro-nacional-de-florestas-publicas

In [ ]:
GLEBA = "GLEBA"

#### Tabular Intersecção

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = GLEBA,
    out_table         = "Tabulate_Inter_CAR_AL_GLEBA",
    class_fields      = "GLEBA",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = GLEBA,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_GLEBA",
    class_fields      = "GLEBA",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = GLEBA,
    out_table         = "Tabulate_Inter_CAR_nALmata_GLEBA",
    class_fields      = "GLEBA",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = GLEBA,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_GLEBA",
    class_fields      = "GLEBA",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = GLEBA,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_GLEBA",
    class_fields      = "GLEBA",
    out_units         = "HECTARES")

#### Ajustar Colunas

##### Adicionar

In [ ]:
Tabulate_Inter_CAR_AL_GLEBA = "Tabulate_Inter_CAR_AL_GLEBA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_AL_GLEBA,
                          "GLEha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_GLEBA = "Tabulate_Inter_CAR_nALcerrado_GLEBA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_GLEBA,
                          "GLEha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALmata_GLEBA = "Tabulate_Inter_CAR_nALmata_GLEBA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_GLEBA,
                          "GLEha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_GLEBA = "Tabulate_Inter_CAR_nALcaatinga_GLEBA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_GLEBA,
                          "GLEha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_GLEBA = "Tabulate_Inter_CAR_nALPamPan_GLEBA"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_GLEBA,
                          "GLEha",
                          "DOUBLE")

##### Calcular

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_GLEBA,
                                "GLEha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_GLEBA,
                                "GLEha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_GLEBA,
                                "GLEha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_GLEBA,
                                "GLEha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_GLEBA,
                                "GLEha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_GLEBA,
    join_field = "cod_imovel", 
    fields     = "GLEha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_GLEBA,
    join_field = "cod_imovel", 
    fields     = "GLEha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_GLEBA,
    join_field = "cod_imovel", 
    fields     = "GLEha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_GLEBA,
    join_field = "cod_imovel", 
    fields     = "GLEha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_GLEBA,
    join_field = "cod_imovel", 
    fields     = "GLEha")

### [TI - FUNAI](https://www.gov.br/funai/pt-br/atuacao/terras-indigenas/geoprocessamento-e-mapas)

FUNAI https://www.gov.br/funai/pt-br/atuacao/terras-indigenas/geoprocessamento-e-mapas

In [ ]:
TI = "TI"

#### Tabular Intersecção

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = TI,
    out_table         = "Tabulate_Inter_CAR_AL_TI",
    class_fields      = "TI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = TI,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_TI",
    class_fields      = "TI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = TI,
    out_table         = "Tabulate_Inter_CAR_nALmata_TI",
    class_fields      = "TI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = TI,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_TI",
    class_fields      = "TI",
    out_units         = "HECTARES")

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = TI,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_TI",
    class_fields      = "TI",
    out_units         = "HECTARES")

#### Ajuste Colunas

##### Adicionar

In [ ]:
Tabulate_Inter_CAR_AL_TI = "Tabulate_Inter_CAR_AL_TI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_AL_TI,
                          "TIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_TI = "Tabulate_Inter_CAR_nALcerrado_TI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_TI,
                          "TIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALmata_TI = "Tabulate_Inter_CAR_nALmata_TI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_TI,
                          "TIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_TI = "Tabulate_Inter_CAR_nALcaatinga_TI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_TI,
                          "TIha",
                          "DOUBLE")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_TI = "Tabulate_Inter_CAR_nALPamPan_TI"

#---------------------------------
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_TI,
                          "TIha",
                          "DOUBLE")

##### Calcular

In [ ]:
Tabulate_Inter_CAR_AL_TI = "Tabulate_Inter_CAR_AL_TI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_TI,
                                "TIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALcerrado_TI = "Tabulate_Inter_CAR_nALcerrado_TI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_TI,
                                "TIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALmata_TI = "Tabulate_Inter_CAR_nALmata_TI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_TI,
                                "TIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_TI = "Tabulate_Inter_CAR_nALcaatinga_TI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_TI,
                                "TIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

In [ ]:
Tabulate_Inter_CAR_nALPamPan_TI = "Tabulate_Inter_CAR_nALPamPan_TI"
#---------------------------------
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_TI,
                                "TIha", 
                                "round(!AREA!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_TI,
    join_field = "cod_imovel", 
    fields     = "TIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_TI,
    join_field = "cod_imovel", 
    fields     = "TIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_TI,
    join_field = "cod_imovel", 
    fields     = "TIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_TI,
    join_field = "cod_imovel", 
    fields     = "TIha")

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_TI,
    join_field = "cod_imovel", 
    fields     = "TIha")

### Null por 0

In [ ]:
fieldObs = arcpy.ListFields(CAR_AL)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_AL, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALPamPan)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALPamPan, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcerrado)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcerrado, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALmata)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALmata, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcaatinga)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcaatinga, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### Sobreposição entre o CAR e Terras Públicas? Sim ou Não!

#### CAR_AL

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["UCha", "TIha", "GLEha", "QUIha"]


campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_AL)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

print(campos_existentessb)
print("---")
print(colunas_presentessb)
print("---")
print(colunas_ausentessb)

In [ ]:
arcpy.management.AddField(CAR_AL,
                          "sobr_tp",
                          "SHORT"
                         )

In [ ]:
if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! >= 1".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_AL,
                                    "sobr_tp",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

#### CAR_nALcerrado

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["UCha", "TIha", "GLEha", "QUIha"]


campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALcerrado)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

print(campos_existentessb)
print("---")
print(colunas_presentessb)
print("---")
print(colunas_ausentessb)

In [ ]:
arcpy.management.AddField(CAR_nALcerrado,
                          "sobr_tp",
                          "SHORT"
                         )

In [ ]:
if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! >= 1".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALcerrado,
                                    "sobr_tp",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

#### CAR_nALmata

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["UCha", "TIha", "GLEha", "QUIha"]


campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALmata)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

print(campos_existentessb)
print("---")
print(colunas_presentessb)
print("---")
print(colunas_ausentessb)

In [ ]:
arcpy.management.AddField(CAR_nALmata,
                          "sobr_tp",
                          "SHORT"
                         )

In [ ]:
if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! >= 1".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALmata,
                                    "sobr_tp",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

#### CAR_nALcaatinga

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["UCha", "TIha", "GLEha", "QUIha"]


campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALcaatinga)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

print(campos_existentessb)
print("---")
print(colunas_presentessb)
print("---")
print(colunas_ausentessb)

In [ ]:
arcpy.management.AddField(CAR_nALcaatinga,
                          "sobr_tp",
                          "SHORT"
                         )

In [ ]:
if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! >= 1".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALcaatinga,
                                    "sobr_tp",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

#### CAR_nALPamPan

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["UCha", "TIha", "GLEha", "QUIha"]


campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALPamPan)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

print(campos_existentessb)
print("---")
print(colunas_presentessb)
print("---")
print(colunas_ausentessb)

In [ ]:
arcpy.management.AddField(CAR_nALPamPan,
                          "sobr_tp",
                          "SHORT"
                         )

In [ ]:
if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! >= 1".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALPamPan,
                                    "sobr_tp",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

### Relacionamento com Informações das Terras Públicas

In [ ]:
CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

UC    = "UC_CNUC_PI"
QUI   = "QUI"
GLEBA = "GLEBA"
TI    = "TI"

#### Dados

In [ ]:
#---------------------------------
arcpy.management.AddField(UC,
                          "TP",
                          "TEXT")

arcpy.management.AddField(UC,
                          "TPid",
                          "TEXT")

arcpy.management.AddField(UC,
                          "TPinfo",
                          "TEXT")

arcpy.management.AddField(UC,
                          "TPfonte",
                          "TEXT")
#---------------------------------
arcpy.management.AddField(TI,
                          "TP",
                          "TEXT")

arcpy.management.AddField(TI,
                          "TPid",
                          "TEXT")

arcpy.management.AddField(TI,
                          "TPinfo",
                          "TEXT")

arcpy.management.AddField(TI,
                          "TPfonte",
                          "TEXT")
#---------------------------------
arcpy.management.AddField(QUI,
                          "TP",
                          "TEXT")

arcpy.management.AddField(QUI,
                          "TPid",
                          "TEXT")

arcpy.management.AddField(QUI,
                          "TPinfo",
                          "TEXT")

arcpy.management.AddField(QUI,
                          "TPfonte",
                          "TEXT")
#---------------------------------
arcpy.management.AddField(GLEBA,
                          "TP",
                          "TEXT")

arcpy.management.AddField(GLEBA,
                          "TPid",
                          "TEXT")

arcpy.management.AddField(GLEBA,
                          "TPinfo",
                          "TEXT")

arcpy.management.AddField(GLEBA,
                          "TPfonte",
                          "TEXT")

In [ ]:
#---------------------------------
arcpy.management.CalculateField(UC,
                                "TP", 
                                "'UNIDADE CONSERVACAO'", 
                                "PYTHON3")

arcpy.management.CalculateField(UC,
                                "TPfonte", 
                                f"'CNUC-GOV e ICMBIO - Acesso: {datetime.datetime.today().strftime('%m/%Y')}'", 
                                "PYTHON3")

arcpy.management.CalculateField(UC,
                                "TPinfo", 
                                "!UCinfo!", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(TI,
                                "TP", 
                                "'TERRITORIO INDIGENA'", 
                                "PYTHON3")

arcpy.management.CalculateField(TI,
                                "TPfonte", 
                                f"'FUNAI - Acesso: {datetime.datetime.today().strftime('%m/%Y')}'", 
                                "PYTHON3")

arcpy.management.CalculateField(TI,
                                "TPinfo", 
                                "!TIinfo!", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(QUI,
                                "TP", 
                                "'QUILOMBOLA'", 
                                "PYTHON3")

arcpy.management.CalculateField(QUI,
                                "TPfonte", 
                                f"'INCRA - Acesso: {datetime.datetime.today().strftime('%m/%Y')}'", 
                                "PYTHON3")

arcpy.management.CalculateField(QUI,
                                "TPinfo", 
                                "!QUIinfo!", 
                                "PYTHON3")

#---------------------------------
arcpy.management.CalculateField(GLEBA,
                                "TP", 
                                "'GLEBA NAO DESTINADA'", 
                                "PYTHON3")

arcpy.management.CalculateField(GLEBA,
                                "TPfonte", 
                                f"'CNFP - Acesso: {datetime.datetime.today().strftime('%m/%Y')}'", 
                                "PYTHON3")

arcpy.management.CalculateField(GLEBA,
                                "TPinfo", 
                                "!GLEBAinfo!", 
                                "PYTHON3")

In [ ]:
with arcpy.da.UpdateCursor(UC, ["TPid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"UC_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
with arcpy.da.UpdateCursor(TI, ["TPid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"TI_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
with arcpy.da.UpdateCursor(GLEBA, ["TPid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"GLEBA_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
with arcpy.da.UpdateCursor(QUI, ["TPid"]) as cursor:
    contador = 1
    for row in cursor:
        row[0] = f"QUI_{contador:03d}" 
        cursor.updateRow(row)
        contador += 1

In [ ]:
arcpy.management.Merge(inputs = [UC,TI,QUI,GLEBA], 
                       output = "TPfinalprj")

In [ ]:
TPfinalprj = "TPfinalprj"

arcpy.management.RepairGeometry(TPfinalprj,
                                delete_null       = "DELETE_NULL",
                                validation_method = "ESRI")

#### AL

In [ ]:
TPfinalprj = "TPfinalprj"

##### Interseccçao

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_AL, TPfinalprj], 
    out_feature_class = "CAR_AL_TP")

##### Dissolve

In [ ]:
statistics_fields = [
    ["TP", "CONCATENATE"],
    ["TPinfo", "CONCATENATE"],
    ["TPfonte", "CONCATENATE"]]

In [ ]:
CAR_AL_TP = "CAR_AL_TP"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_AL_TP,
    out_feature_class = "CAR_AL_TP_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Cerrado

##### Interseccçao

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALcerrado, TPfinalprj], 
    out_feature_class = "CAR_nALcerrado_TP")

##### Dissolve

In [ ]:
CAR_nALcerrado_TP = "CAR_nALcerrado_TP"

statistics_fields = [
    ["TP", "CONCATENATE"],
    ["TPinfo", "CONCATENATE"],
    ["TPfonte", "CONCATENATE"]]

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALcerrado_TP,
    out_feature_class = "CAR_nALcerrado_TP_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Mata

##### Interseccçao

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALmata, TPfinalprj], 
    out_feature_class = "CAR_nALmata_TP")

##### Dissolve

In [ ]:
CAR_nALmata_TP = "CAR_nALmata_TP"

statistics_fields = [
    ["TP", "CONCATENATE"],
    ["TPinfo", "CONCATENATE"],
    ["TPfonte", "CONCATENATE"]]

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALmata_TP,
    out_feature_class = "CAR_nALmata_TP_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Caatinga

##### Interseccçao

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALcaatinga, TPfinalprj], 
    out_feature_class = "CAR_nALcaatinga_TP")

##### Dissolve

In [ ]:
CAR_nALcaatinga_TP = "CAR_nALcaatinga_TP"

statistics_fields = [
    ["TP", "CONCATENATE"],
    ["TPinfo", "CONCATENATE"],
    ["TPfonte", "CONCATENATE"]]

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALcaatinga_TP,
    out_feature_class = "CAR_nALcaatinga_TP_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### PamPan

##### Interseccçao

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALPamPan, TPfinalprj], 
    out_feature_class = "CAR_nALPamPan_TP")

##### Dissolve

In [ ]:
CAR_nALPamPan_TP = "CAR_nALPamPan_TP"

statistics_fields = [
    ["TP", "CONCATENATE"],
    ["TPinfo", "CONCATENATE"],
    ["TPfonte", "CONCATENATE"]]

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALPamPan_TP,
    out_feature_class = "CAR_nALPamPan_TP_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Merge Final e Export

In [ ]:
CAR_AL_TP          = "CAR_AL_TP_dissolve"
CAR_nALcerrado_TP  = "CAR_nALcerrado_TP_dissolve"
CAR_nALmata_TP     = "CAR_nALmata_TP_dissolve"
CAR_nALcaatinga_TP = "CAR_nALcaatinga_TP_dissolve"
CAR_nALPamPan_TP   = "CAR_nALPamPan_TP_dissolve"


arcpy.management.Merge([CAR_AL_TP, CAR_nALcerrado_TP, CAR_nALmata_TP, CAR_nALcaatinga_TP, CAR_nALPamPan_TP], 
                       "CAR_TERRASPUBLICASinfo")

In [ ]:
CAR_TERRASPUBLICASinfo = "CAR_TERRASPUBLICASinfo"

arcpy.management.CalculateField(
    in_table        = CAR_TERRASPUBLICASinfo,
    field           = "CONCATENATE_TPfonte",
    expression      = "clean_value(!CONCATENATE_TPfonte!)",
    expression_type = "PYTHON3",
    code_block="""
def clean_value(value):
    if not value:
        return ""
    items = [item.strip() for item in value.split(";") if item.strip()]  # Remove espaços extras
    unique_items = sorted(set(items))  # Remove duplicatas e ordena
    return "; ".join(unique_items)  # Adiciona espaço após ";"
"""
)

In [ ]:
arcpy.management.CalculateField(
    in_table        = CAR_TERRASPUBLICASinfo,
    field           = "CONCATENATE_TP",
    expression      = "clean_value(!CONCATENATE_TP!)",
    expression_type = "PYTHON3",
    code_block="""
def clean_value(value):
    if not value:
        return ""
    items = [item.strip() for item in value.split(";") if item.strip()]  # Remove espaços extras
    unique_items = sorted(set(items))  # Remove duplicatas e ordena
    return "; ".join(unique_items)  # Adiciona espaço após ";"
"""
)

In [ ]:
arcpy.management.CalculateField(
    in_table        = CAR_TERRASPUBLICASinfo,
    field           = "CONCATENATE_TPinfo",
    expression      = "clean_value(!CONCATENATE_TPinfo!)",
    expression_type = "PYTHON3",
    code_block="""
def clean_value(value):
    if not value:
        return ""
    items = [item.strip() for item in value.split(";") if item.strip()]  # Remove espaços extras
    unique_items = sorted(set(items))  # Remove duplicatas e ordena
    return "; ".join(unique_items)  # Adiciona espaço após ";"
"""
)

## Sobreposição Embargos

### <mark>Onde Estamos?</mark>

In [ ]:
EMBfinalprj     = "EMBfinalprj"

CAR_AL          = "CAR_AL"
CAR_nALcerrado  = "CAR_nALcerrado"
CAR_nALmata     = "CAR_nALmata"
CAR_nALcaatinga = "CAR_nALcaatinga"
CAR_nALPamPan   = "CAR_nALPamPan"

EMBIBprj = "EMBIBprj"
EMBICprj = "EMBICprj"
EMBMTprj = "EMBMTprj"
EMBPAprj = "EMBPAprj"
EMBGOprj = "EMBGOprj"
EMBAMprj = "EMBAMprj"

### CAR AL

#### Tabular Intersecção 

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_AL,
    zone_fields       = "cod_imovel",
    in_class_features = EMBfinalprj,
    out_table         = "Tabulate_Inter_CAR_AL_EMB",
    class_fields      = "EMBfonte",
    out_units         = "HECTARES")

#### Pivot Table

In [ ]:
Tabulate_Inter_CAR_AL_EMB = "Tabulate_Inter_CAR_AL_EMB"

In [ ]:
arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_AL_EMB, 
    fields      = "cod_imovel", 
    pivot_field = "EMBfonte", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_AL_EMB_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_AL_EMB = "Tabulate_Inter_CAR_AL_EMB_pivot"

In [ ]:
Tabulate_Inter_CAR_AL_EMB = "Tabulate_Inter_CAR_AL_EMB_pivot"

fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_AL_EMB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_AL_EMB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustar Colunas

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBIBha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBICha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBMTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBPAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBGOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_AL_EMB,
                          "EMBAMha",
                          "DOUBLE")

In [ ]:
fields = arcpy.ListFields(Tabulate_Inter_CAR_AL_EMB)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBIBha", 
                                "round(!IBAMA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBICha", 
                                "round(!ICMBIO___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBMTha", 
                                "round(!SEMA_MT___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBPAha", 
                                "round(!SEMA_PA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBGOha", 
                                "round(!SEMA_GO___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_AL_EMB,
                                "EMBAMha", 
                                "round(!IPAAM_AM___28_01_2025!,4)", 
                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_AL, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_AL_EMB,
    join_field = "cod_imovel", 
    fields     = ["EMBIBha","EMBICha","EMBMTha","EMBPAha","EMBGOha","EMBAMha"])

In [ ]:
fieldObs = arcpy.ListFields(CAR_AL)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_AL, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### CAR nAL - Cerrado

#### Tabular Intersecção 

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcerrado,
    zone_fields       = "cod_imovel",
    in_class_features = EMBfinalprj,
    out_table         = "Tabulate_Inter_CAR_nALcerrado_EMB",
    class_fields      = "EMBfonte",
    out_units         = "HECTARES")

#### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALcerrado_EMB = "Tabulate_Inter_CAR_nALcerrado_EMB"

arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcerrado_EMB, 
    fields      = "cod_imovel", 
    pivot_field = "EMBfonte", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcerrado_EMB_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALcerrado_EMB = "Tabulate_Inter_CAR_nALcerrado_EMB_pivot"

fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcerrado_EMB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcerrado_EMB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustar Colunas

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBIBha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBICha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBMTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBPAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBGOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcerrado_EMB,
                          "EMBAMha",
                          "DOUBLE")

In [ ]:
fields = arcpy.ListFields(Tabulate_Inter_CAR_nALcerrado_EMB)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
                                "EMBIBha", 
                                "round(!IBAMA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
                                "EMBICha", 
                                "round(!ICMBIO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
#                                "EMBMTha", 
#                                "round(!SEMA_MT___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
#                                "EMBPAha", 
#                                "round(!SEMA_PA___28_01_2025!,4)", 
#                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
                                "EMBGOha", 
                                "round(!SEMA_GO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcerrado_EMB,
#                                "EMBAMha", 
#                                "round(!IPAAM_AM___28_01_2025!,4)", 
#                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcerrado, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcerrado_EMB,
    join_field = "cod_imovel", 
    fields     = ["EMBIBha","EMBICha","EMBMTha","EMBPAha","EMBGOha","EMBAMha"])

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcerrado)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcerrado, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### CAR_nALmata

#### Tabular Intersecção 

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALmata,
    zone_fields       = "cod_imovel",
    in_class_features = EMBfinalprj,
    out_table         = "Tabulate_Inter_CAR_nALmata_EMB",
    class_fields      = "EMBfonte",
    out_units         = "HECTARES")

#### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALmata_EMB = "Tabulate_Inter_CAR_nALmata_EMB"

arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALmata_EMB, 
    fields      = "cod_imovel", 
    pivot_field = "EMBfonte", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALmata_EMB_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALmata_EMB = "Tabulate_Inter_CAR_nALmata_EMB_pivot"

fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALmata_EMB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALmata_EMB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustar Colunas

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBIBha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBICha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBMTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBPAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBGOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALmata_EMB,
                          "EMBAMha",
                          "DOUBLE")

In [ ]:
fields = arcpy.ListFields(Tabulate_Inter_CAR_nALmata_EMB)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
                                "EMBIBha", 
                                "round(!IBAMA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
                                "EMBICha", 
                                "round(!ICMBIO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
#                                "EMBMTha", 
#                                "round(!SEMA_MT___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
#                                "EMBPAha", 
#                                "round(!SEMA_PA___28_01_2025!,4)", 
#                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
                                "EMBGOha", 
                                "round(!SEMA_GO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALmata_EMB,
#                                "EMBAMha", 
#                                "round(!IPAAM_AM___28_01_2025!,4)", 
#                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALmata, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALmata_EMB,
    join_field = "cod_imovel", 
    fields     = ["EMBIBha","EMBICha","EMBMTha","EMBPAha","EMBGOha","EMBAMha"])

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALmata)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALmata, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### CAR_nALcaatinga

#### Tabular Intersecção 

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALcaatinga,
    zone_fields       = "cod_imovel",
    in_class_features = EMBfinalprj,
    out_table         = "Tabulate_Inter_CAR_nALcaatinga_EMB",
    class_fields      = "EMBfonte",
    out_units         = "HECTARES")

#### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_EMB = "Tabulate_Inter_CAR_nALcaatinga_EMB"

arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALcaatinga_EMB, 
    fields      = "cod_imovel", 
    pivot_field = "EMBfonte", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALcaatinga_EMB_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALcaatinga_EMB = "Tabulate_Inter_CAR_nALcaatinga_EMB_pivot"

fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALcaatinga_EMB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALcaatinga_EMB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustar Colunas

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBIBha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBICha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBMTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBPAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBGOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                          "EMBAMha",
                          "DOUBLE")

In [ ]:
fields = arcpy.ListFields(Tabulate_Inter_CAR_nALcaatinga_EMB)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                                "EMBIBha", 
                                "round(!IBAMA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
                                "EMBICha", 
                                "round(!ICMBIO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
#                                "EMBMTha", 
#                                "round(!SEMA_MT___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
#                                "EMBPAha", 
#                                "round(!SEMA_PA___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
#                                "EMBGOha", 
#                                "round(!SEMA_GO___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALcaatinga_EMB,
#                                "EMBAMha", 
#                                "round(!IPAAM_AM___28_01_2025!,4)", 
#                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALcaatinga, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALcaatinga_EMB,
    join_field = "cod_imovel", 
    fields     = ["EMBIBha","EMBICha","EMBMTha","EMBPAha","EMBGOha","EMBAMha"])

In [ ]:
fieldObs = arcpy.ListFields(CAR_nALcaatinga)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALcaatinga, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### CAR_nALPamPan

#### Tabular Intersecção 

In [ ]:
arcpy.analysis.TabulateIntersection(
    in_zone_features  = CAR_nALPamPan,
    zone_fields       = "cod_imovel",
    in_class_features = EMBfinalprj,
    out_table         = "Tabulate_Inter_CAR_nALPamPan_EMB",
    class_fields      = "EMBfonte",
    out_units         = "HECTARES")

#### Pivot Table

In [ ]:
Tabulate_Inter_CAR_nALPamPan_EMB = "Tabulate_Inter_CAR_nALPamPan_EMB"

arcpy.management.PivotTable(
    in_table    = Tabulate_Inter_CAR_nALPamPan_EMB, 
    fields      = "cod_imovel", 
    pivot_field = "EMBfonte", 
    value_field = "AREA", 
    out_table   = "Tabulate_Inter_CAR_nALPamPan_EMB_pivot")

#### Null por 0

In [ ]:
Tabulate_Inter_CAR_nALPamPan_EMB = "Tabulate_Inter_CAR_nALPamPan_EMB_pivot"

fieldObs = arcpy.ListFields(Tabulate_Inter_CAR_nALPamPan_EMB)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(Tabulate_Inter_CAR_nALPamPan_EMB, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

#### Ajustar Colunas

In [ ]:
arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBIBha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBICha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBMTha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBPAha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBGOha",
                          "DOUBLE")

arcpy.management.AddField(Tabulate_Inter_CAR_nALPamPan_EMB,
                          "EMBAMha",
                          "DOUBLE")

In [ ]:
fields = arcpy.ListFields(Tabulate_Inter_CAR_nALPamPan_EMB)

for field in fields:
    print(f"Nome: {field.name}, Tipo: {field.type}")

In [ ]:
arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
                                "EMBIBha", 
                                "round(!IBAMA___28_01_2025!,4)", 
                                "PYTHON3")

arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
                                "EMBICha", 
                                "round(!ICMBIO___28_01_2025!,4)", 
                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
#                                "EMBMTha", 
#                                "round(!SEMA_MT___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
#                                "EMBPAha", 
#                                "round(!SEMA_PA___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
#                                "EMBGOha", 
#                                "round(!SEMA_GO___28_01_2025!,4)", 
#                                "PYTHON3")

#arcpy.management.CalculateField(Tabulate_Inter_CAR_nALPamPan_EMB,
#                                "EMBAMha", 
#                                "round(!IPAAM_AM___28_01_2025!,4)", 
#                                "PYTHON3")

#### Join

In [ ]:
arcpy.management.JoinField(
    in_data    = CAR_nALPamPan, 
    in_field   = "cod_imovel", 
    join_table = Tabulate_Inter_CAR_nALPamPan_EMB,
    join_field = "cod_imovel", 
    fields     = ["EMBIBha","EMBICha","EMBMTha","EMBPAha","EMBGOha","EMBAMha"])


In [ ]:
fieldObs = arcpy.ListFields(CAR_nALPamPan)  
fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames)  

with arcpy.da.UpdateCursor(CAR_nALPamPan, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0.0"  
      
      
        curU.updateRow(rowU)

del curU

### Sobreposição entre o CAR e Embargos? Sim ou Não!

In [ ]:
arcpy.management.AddField(CAR_AL,
                          "sobr_emb",
                          "SHORT"
                         )

arcpy.management.AddField(CAR_nALcerrado,
                          "sobr_emb",
                          "SHORT"
                         )

arcpy.management.AddField(CAR_nALmata,
                          "sobr_emb",
                          "SHORT"
                         )

arcpy.management.AddField(CAR_nALcaatinga,
                          "sobr_emb",
                          "SHORT"
                         )

arcpy.management.AddField(CAR_nALPamPan,
                          "sobr_emb",
                          "SHORT"
                         )

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["EMBIBha", "EMBICha", "EMBMTha", "EMBPAha", "EMBAMha", "EMBGOha"]
campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_AL)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! > 0".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_AL,
                                    "sobr_emb",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["EMBIBha", "EMBICha", "EMBMTha", "EMBPAha", "EMBAMha", "EMBGOha"]
campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALcerrado)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! > 0".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALcerrado,
                                    "sobr_emb",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["EMBIBha", "EMBICha", "EMBMTha", "EMBPAha", "EMBAMha", "EMBGOha"]
campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALmata)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! > 0".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALmata,
                                    "sobr_emb",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["EMBIBha", "EMBICha", "EMBMTha", "EMBPAha", "EMBAMha", "EMBGOha"]
campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALcaatinga)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! > 0".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALcaatinga,
                                    "sobr_emb",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

In [ ]:
# Sobreposição Terras Publicas (1=sim / 0=nao)
colunassb = ["EMBIBha", "EMBICha", "EMBMTha", "EMBPAha", "EMBAMha", "EMBGOha"]
campos_existentessb = [campo.name for campo in arcpy.ListFields(CAR_nALPamPan)]
colunas_presentessb = [coluna for coluna in colunassb if coluna in campos_existentessb]
colunas_ausentessb  = [coluna for coluna in colunassb if coluna not in campos_existentessb]

if colunas_presentessb:
    # Expressão para o cálculo de "sobr_tp"
    expressao = "1 if ({}) else 0".format(" or ".join(["!{}! > 0".format(coluna) for coluna in colunas_presentessb]))

    arcpy.management.CalculateField(CAR_nALPamPan,
                                    "sobr_emb",
                                    expressao,
                                    "PYTHON3"
                                    )

    print("Colunas presentes:")
    for coluna in colunas_presentessb:
        print(coluna)
    print("Fórmula utilizada:")
    print(expressao)
else:
    print("Nenhuma das colunas necessárias está presente na tabela.")

print("Colunas ausentes:")
for coluna in colunas_ausentessb:
    print(coluna)

### Relacionamento com Informações dos Embargos

#### AL

##### Interseccão

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_AL, EMBfinalprj], 
    out_feature_class = "CAR_AL_EMB")

##### Dissolve

In [ ]:
statistics_fields = [
    ["EMBinfo", "CONCATENATE"],
    ["EMBfonte", "CONCATENATE"]]

In [ ]:
CAR_AL_EMB = "CAR_AL_EMB"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_AL_EMB, 
    out_feature_class = "CAR_AL_EMB_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Cerrado

##### Interseccão

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALcerrado, EMBfinalprj], 
    out_feature_class = "CAR_nALcerrado_EMB")

##### Dissolve

In [ ]:
statistics_fields = [
    ["EMBinfo", "CONCATENATE"],
    ["EMBfonte", "CONCATENATE"]]

In [ ]:
CAR_nALcerrado_EMB = "CAR_nALcerrado_EMB"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALcerrado_EMB,
    out_feature_class = "CAR_nALcerrado_EMB_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Mata

##### Interseccão

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALmata, EMBfinalprj], 
    out_feature_class = "CAR_nALmata_EMB")

##### Dissolve

In [ ]:
statistics_fields = [
    ["EMBinfo", "CONCATENATE"],
    ["EMBfonte", "CONCATENATE"]]

In [ ]:
CAR_nALmata_EMB = "CAR_nALmata_EMB"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALmata_EMB, 
    out_feature_class = "CAR_nALmata_EMB_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Caatinga

##### Interseccão

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALcaatinga, EMBfinalprj], 
    out_feature_class = "CAR_nALcaatinga_EMB")

##### Dissolve

In [ ]:
statistics_fields = [
    ["EMBinfo", "CONCATENATE"],
    ["EMBfonte", "CONCATENATE"]]

In [ ]:
CAR_nALcaatinga_EMB = "CAR_nALcaatinga_EMB"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALcaatinga_EMB, 
    out_feature_class = "CAR_nALcaatinga_EMB_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### PamPan

##### Interseccão

In [ ]:
arcpy.analysis.PairwiseIntersect(
    in_features       = [CAR_nALPamPan, EMBfinalprj], 
    out_feature_class = "CAR_nALPamPan_EMB")

##### Dissolve

In [ ]:
statistics_fields = [
    ["EMBinfo", "CONCATENATE"],
    ["EMBfonte", "CONCATENATE"]]

In [ ]:
CAR_nALPamPan = "CAR_nALPamPan_EMB"

arcpy.analysis.PairwiseDissolve(
    in_features       = CAR_nALPamPan, 
    out_feature_class = "CAR_nALPamPan_EMB_dissolve", 
    dissolve_field    = "cod_imovel", 
    statistics_fields = statistics_fields,
    multi_part        = "MULTI_PART", #"SINGLE_PART" 
    concatenation_separator = "; "
)

#### Merge Final e Export

In [ ]:
CAR_AL_EMB          = "CAR_AL_EMB_dissolve"
CAR_nALcerrado_EMB  = "CAR_nALcerrado_EMB_dissolve"
CAR_nALmata_EMB     = "CAR_nALmata_EMB_dissolve"
CAR_nALcaatinga_EMB = "CAR_nALcaatinga_EMB_dissolve"
CAR_nALPamPan_EMB   = "CAR_nALPamPan_EMB_dissolve"


arcpy.management.Merge([CAR_AL_EMB, CAR_nALcerrado_EMB, CAR_nALmata_EMB, CAR_nALcaatinga_EMB, CAR_nALPamPan_EMB], 
                       "CAR_EMBARGOSinfo")

In [ ]:
arcpy.management.CalculateField(
    in_table        = CAR_EMBARGOSinfo,
    field           = "CONCATENATE_EMBfonte",
    expression      = "clean_value(!CONCATENATE_EMBfonte!)",
    expression_type = "PYTHON3",
    code_block="""
def clean_value(value):
    if not value:
        return ""
    items = [item.strip() for item in value.split(";") if item.strip()]  # Remove espaços extras
    unique_items = sorted(set(items))  # Remove duplicatas e ordena
    return "; ".join(unique_items)  # Adiciona espaço após ";"
"""
)

## Comentarios Geo

In [ ]:
fields = arcpy.ListFields(CAR_AL)

for field in fields:
    print(field.name)

In [ ]:
arcpy.AddField_management(CAR_AL,          "GEO", "TEXT")
arcpy.AddField_management(CAR_nALcerrado,  "GEO", "TEXT")
arcpy.AddField_management(CAR_nALmata,     "GEO", "TEXT")
arcpy.AddField_management(CAR_nALcaatinga, "GEO", "TEXT")
arcpy.AddField_management(CAR_nALPamPan,   "GEO", "TEXT")

In [ ]:
arcpy.AddField_management(CAR_AL,          "GEOC", "TEXT")
arcpy.AddField_management(CAR_nALcerrado,  "GEOC", "TEXT")
arcpy.AddField_management(CAR_nALmata,     "GEOC", "TEXT")
arcpy.AddField_management(CAR_nALcaatinga, "GEOC", "TEXT")
arcpy.AddField_management(CAR_nALPamPan,   "GEOC", "TEXT")

In [ ]:
# Lista de colunas que indicam desmatamento entre 2020 e 2024
desmatamento_cols = ["P2020", "P2021", "P2022", "P2023", "P2024", "A2024"]

# Atualizar a nova coluna "GEOC" com base nas condições
with arcpy.da.UpdateCursor(CAR_AL, 
                           ["sobr_tp", "sobr_emb", "P08_19"] + desmatamento_cols + ["EVNha", "GEOC", "GEO"]) as cursor:
    
    for row in cursor:
        classificacoes = []  # Lista para armazenar as classificações
        
        # Verificar sobreposições e embargos
        if row[0] == 1:
            classificacoes.append("Public Land Overlap")
        if row[1] == 1:
            classificacoes.append("Embargoed Area")
        
        # Verificar desmatamento 2008/2019
        if row[2] != 0:
            classificacoes.append("2008/2019 Deforestation")
        
        # Verificar desmatamento 2020/2024
        for i in range(3, 9):  # Índices das colunas P2020 a A2024
            if row[i] != 0:
                classificacoes.append("2020/2024 Deforestation")
                break  # Já encontramos um desmatamento recente, então saímos do loop
        
        # Verificar déficit de vegetação nativa
        if row[9] < 0:
            classificacoes.append("NV Deficit")
        
        # Concatenar as classificações e atualizar a coluna GEOC
        row[10] = "; ".join(classificacoes) if classificacoes else "Ok"
        
        # Definir o valor da coluna GEO
        row[11] = "Ok" if row[10] == "Ok" else "Alert!"
        
        cursor.updateRow(row)

In [ ]:
# Lista de colunas que indicam desmatamento entre 2020 e 2024
desmatamento_cols = ["P2020", "P2021", "P2022", "P2023", "P2024", "A2024"]

# Atualizar a nova coluna "GEOC" com base nas condições
with arcpy.da.UpdateCursor(CAR_nALcerrado, 
                           ["sobr_tp", "sobr_emb", "P08_19"] + desmatamento_cols + ["EVNha", "GEOC", "GEO"]) as cursor:
    
    for row in cursor:
        classificacoes = []  # Lista para armazenar as classificações
        
        # Verificar sobreposições e embargos
        if row[0] == 1:
            classificacoes.append("Public Land Overlap")
        if row[1] == 1:
            classificacoes.append("Embargoed Area")
        
        # Verificar desmatamento 2008/2019
        if row[2] != 0:
            classificacoes.append("2008/2019 Deforestation")
        
        # Verificar desmatamento 2020/2024
        for i in range(3, 9):  # Índices das colunas P2020 a A2024
            if row[i] != 0:
                classificacoes.append("2020/2024 Deforestation")
                break  # Já encontramos um desmatamento recente, então saímos do loop
        
        # Verificar déficit de vegetação nativa
        if row[9] < 0:
            classificacoes.append("NV Deficit")
        
        # Concatenar as classificações e atualizar a coluna GEOC
        row[10] = "; ".join(classificacoes) if classificacoes else "Ok"
        
        # Definir o valor da coluna GEO
        row[11] = "Ok" if row[10] == "Ok" else "Alert!"
        
        cursor.updateRow(row)

In [ ]:
# Lista de colunas que indicam desmatamento entre 2020 e 2024
desmatamento_cols = ["P2020", "P2021", "P2022", "P2023", "P2024", "A2024"]

# Atualizar a nova coluna "GEOC" com base nas condições
with arcpy.da.UpdateCursor(CAR_nALmata, 
                           ["sobr_tp", "sobr_emb", "P08_19"] + desmatamento_cols + ["EVNha", "GEOC", "GEO"]) as cursor:
    
    for row in cursor:
        classificacoes = []  # Lista para armazenar as classificações
        
        # Verificar sobreposições e embargos
        if row[0] == 1:
            classificacoes.append("Public Land Overlap")
        if row[1] == 1:
            classificacoes.append("Embargoed Area")
        
        # Verificar desmatamento 2008/2019
        if row[2] != 0:
            classificacoes.append("2008/2019 Deforestation")
        
        # Verificar desmatamento 2020/2024
        for i in range(3, 9):  # Índices das colunas P2020 a A2024
            if row[i] != 0:
                classificacoes.append("2020/2024 Deforestation")
                break  # Já encontramos um desmatamento recente, então saímos do loop
        
        # Verificar déficit de vegetação nativa
        if row[9] < 0:
            classificacoes.append("NV Deficit")
        
        # Concatenar as classificações e atualizar a coluna GEOC
        row[10] = "; ".join(classificacoes) if classificacoes else "Ok"
        
        # Definir o valor da coluna GEO
        row[11] = "Ok" if row[10] == "Ok" else "Alert!"
        
        cursor.updateRow(row)

In [ ]:
# Lista de colunas que indicam desmatamento entre 2020 e 2024
desmatamento_cols = ["P2020", "P2021", "P2022", "P2023", "P2024", "A2024"]

# Atualizar a nova coluna "GEOC" com base nas condições
with arcpy.da.UpdateCursor(CAR_nALcaatinga, 
                           ["sobr_tp", "sobr_emb", "P08_19"] + desmatamento_cols + ["EVNha", "GEOC", "GEO"]) as cursor:
    
    for row in cursor:
        classificacoes = []  # Lista para armazenar as classificações
        
        # Verificar sobreposições e embargos
        if row[0] == 1:
            classificacoes.append("Public Land Overlap")
        if row[1] == 1:
            classificacoes.append("Embargoed Area")
        
        # Verificar desmatamento 2008/2019
        if row[2] != 0:
            classificacoes.append("2008/2019 Deforestation")
        
        # Verificar desmatamento 2020/2024
        for i in range(3, 9):  # Índices das colunas P2020 a A2024
            if row[i] != 0:
                classificacoes.append("2020/2024 Deforestation")
                break  # Já encontramos um desmatamento recente, então saímos do loop
        
        # Verificar déficit de vegetação nativa
        if row[9] < 0:
            classificacoes.append("NV Deficit")
        
        # Concatenar as classificações e atualizar a coluna GEOC
        row[10] = "; ".join(classificacoes) if classificacoes else "Ok"
        
        # Definir o valor da coluna GEO
        row[11] = "Ok" if row[10] == "Ok" else "Alert!"
        
        cursor.updateRow(row)

In [ ]:
# Lista de colunas que indicam desmatamento entre 2020 e 2024
desmatamento_cols = ["P2020", "P2021", "P2022", "P2023", "P2024", "A2024"]

# Atualizar a nova coluna "GEOC" com base nas condições
with arcpy.da.UpdateCursor(CAR_nALPamPan, 
                           ["sobr_tp", "sobr_emb", "P08_19"] + desmatamento_cols + ["EVNha", "GEOC", "GEO"]) as cursor:
    
    for row in cursor:
        classificacoes = []  # Lista para armazenar as classificações
        
        # Verificar sobreposições e embargos
        if row[0] == 1:
            classificacoes.append("Public Land Overlap")
        if row[1] == 1:
            classificacoes.append("Embargoed Area")
        
        # Verificar desmatamento 2008/2019
        if row[2] != 0:
            classificacoes.append("2008/2019 Deforestation")
        
        # Verificar desmatamento 2020/2024
        for i in range(3, 9):  # Índices das colunas P2020 a A2024
            if row[i] != 0:
                classificacoes.append("2020/2024 Deforestation")
                break  # Já encontramos um desmatamento recente, então saímos do loop
        
        # Verificar déficit de vegetação nativa
        if row[9] < 0:
            classificacoes.append("NV Deficit")
        
        # Concatenar as classificações e atualizar a coluna GEOC
        row[10] = "; ".join(classificacoes) if classificacoes else "Ok"
        
        # Definir o valor da coluna GEO
        row[11] = "Ok" if row[10] == "Ok" else "Alert!"
        
        cursor.updateRow(row)